### Execution Environment

This notebook is designed to run both **locally** and in **Google Colab**.

When executed in Google Colab, the notebook automatically uses the project
directory located at:

`/content/drive/MyDrive/pads`

When executed locally, the notebook assumes it is started from the cloned
project root directory (`pads/`) and automatically uses that path as the
project root. Alternatively, the project root can be specified explicitly via
the `PADS_ROOT` environment variable.

No manual path changes are required when these conventions are followed.

In [1]:
# Install required packages
print("Installing required packages...")
!pip install torch torchvision torchaudio transformers datasets scikit-learn pandas numpy matplotlib seaborn tqdm wfdb onnx onnxruntime onnxruntime-tools onnxscript -q

Installing required packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 152.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [17]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import requests
import zipfile
from tqdm import tqdm
import struct
import itertools
import inspect
import re
import io
import ast
import copy
import time
import random
import gc
import glob


# Import PyTorch and related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.ao.quantization as aoq

from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, balanced_accuracy_score, accuracy_score, precision_score, f1_score, recall_score

from transformers import PretrainedConfig, PreTrainedModel

import onnxruntime as ort
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

from IPython.display import display

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# --- Paths / project root (Colab + local) ---
def _detect_base_dir() -> Path:
    """
    Priority:
      1) Use PADS_ROOT env var if set (works for local + Colab).
      2) If running in Colab and Drive-mounted path exists -> use it.
      3) Otherwise use the current working directory (assume repo root).
    """
    env_root = os.environ.get("PADS_ROOT")
    if env_root:
        return Path(env_root).expanduser().resolve()

    colab_drive_root = Path("/content/drive/MyDrive/pads")
    if colab_drive_root.exists():
        return colab_drive_root.resolve()

    return Path.cwd().resolve()

BASE_DIR = _detect_base_dir()

# Define subdirectories
DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"
RESULTS_DIR = BASE_DIR / "results"
MOVEMENT_DIR = BASE_DIR / "preprocessed" / "movement"
PATIENTS_DIR = BASE_DIR / "patients"
QUESTIONNAIRE_DIR = BASE_DIR / "questionnaire"

# Define specific file paths
LABELS_CSV = BASE_DIR / "preprocessed" / "file_list.csv"

# (Optional) Create output directories if they don't exist
for directory in [MODELS_DIR, RESULTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# (Optional) Sanity checks for required inputs
required = {
    "MOVEMENT_DIR": MOVEMENT_DIR,
    "LABELS_CSV": LABELS_CSV,
    "PATIENTS_DIR": PATIENTS_DIR,
    "QUESTIONNAIRE_DIR": QUESTIONNAIRE_DIR,
}
missing = [name for name, p in required.items() if not p.exists()]
if missing:
    raise FileNotFoundError(
        "Missing required data paths under BASE_DIR:\n"
        + "\n".join([f"- {name}: {required[name]}" for name in missing])
        + "\n\nIf running locally, start Jupyter from the repo root (pads/) "
          "or set PADS_ROOT to that path."
    )

print("Directory structure and constants defined successfully!")
print(f"BASE_DIR: {BASE_DIR}")
print(f"DATA_DIR: {DATA_DIR}")
print(f"MODELS_DIR: {MODELS_DIR}")
print(f"RESULTS_DIR: {RESULTS_DIR}")
print(f"MOVEMENT_DIR: {MOVEMENT_DIR}")
print(f"PATIENTS_DIR: {PATIENTS_DIR}")
print(f"QUESTIONNAIRE_DIR: {QUESTIONNAIRE_DIR}")
print(f"LABELS_CSV: {LABELS_CSV}")

Directory structure and constants defined successfully!
BASE_DIR: /content/drive/MyDrive/pads
DATA_DIR: /content/drive/MyDrive/pads/data
MODELS_DIR: /content/drive/MyDrive/pads/models
RESULTS_DIR: /content/drive/MyDrive/pads/results
MOVEMENT_DIR: /content/drive/MyDrive/pads/preprocessed/movement
PATIENTS_DIR: /content/drive/MyDrive/pads/patients
QUESTIONNAIRE_DIR: /content/drive/MyDrive/pads/questionnaire
LABELS_CSV: /content/drive/MyDrive/pads/preprocessed/file_list.csv


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Download PADS dataset
#def download_pads_dataset():
#    """Download PADS dataset from PhysioNet"""
#    dataset_url = "https://physionet.org/files/parkinsons-disease-smartwatch/1.0.0/"

#    print("Downloading PADS dataset...")
#    print("Note: This is a large dataset (~1.4GB). Please be patient.")

#    # Use wget to download the entire dataset
#    download_cmd = f"wget -r -N -c -np -P {DATA_DIR} {dataset_url}"

#    try:
#        os.system(download_cmd)
#        print("Dataset download completed!")

        # Check if download was successful
#        expected_path = DATA_DIR / "physionet.org" / "files" / "parkinsons-disease-smartwatch" / "1.0.0"
#        if expected_path.exists():
#            print(f"Dataset found at: {expected_path}")
#            return expected_path
#        else:
#            print("Download may not be complete. Checking alternative paths...")
#            return None

#    except Exception as e:
#        print(f"Error downloading dataset: {e}")
#        print("Please manually download from: https://physionet.org/content/parkinsons-disease-smartwatch/1.0.0/")
#        return None

# Alternative: Direct file download using requests
#def download_file_direct(url, filepath):
#    """Download a single file with progress bar"""
#    response = requests.get(url, stream=True)
#    total_size = int(response.headers.get('content-length', 0))

#    with open(filepath, 'wb') as file, tqdm(
#        desc=filepath.name,
#        total=total_size,
#        unit='B',
#        unit_scale=True,
#        unit_divisor=1024,
#    ) as pbar:
#        for chunk in response.iter_content(chunk_size=8192):
#            size = file.write(chunk)
#            pbar.update(size)
#
# Try to download the dataset
#dataset_path = download_pads_dataset()

Installing required packages...
Using device: cuda


## Framework 1: (Modality-based)

### Data Cleaning and Preparation Pipeline

In [6]:
class CleanDataPipeline:
    def __init__(self, movement_dir: Path, patients_dir: Path, questionnaire_dir: Path):
        self.preprocessed_path = Path(movement_dir)
        self.patients_path = Path(patients_dir)
        self.questionnaire_path = Path(questionnaire_dir)

        self._analyze_bin_structure()
        self.patient_metadata = self._load_patient_metadata()
        self.questionnaire_data = self._load_questionnaire_data()

    def _analyze_bin_structure(self):
        bin_files = list(self.preprocessed_path.glob("*.bin"))
        if len(bin_files) == 0:
            raise ValueError("No .bin files found!")

        sample_file = bin_files[0]
        file_size = sample_file.stat().st_size
        num_values = file_size // 4
        print(f"Found {len(bin_files)} .bin files")

        if num_values == 128832:
            self.actual_timesteps = 976
            self.actual_channels = 132
        else:
            self.actual_channels = 132
            self.actual_timesteps = num_values // 132

        print(f"Dimensions: {self.actual_timesteps} × {self.actual_channels}")

    def load_bin_file_raw(self, patient_id, target_length=950):
        patient_id_str = f"{patient_id:03d}_ml.bin"
        bin_file = self.preprocessed_path / patient_id_str

        if not bin_file.exists():
            return None

        try:
            with open(bin_file, 'rb') as f:
                data = np.frombuffer(f.read(), dtype=np.float32)

                if len(data) == self.actual_channels * self.actual_timesteps:
                    data = data.reshape(self.actual_timesteps, self.actual_channels)
                    data = self._adjust_to_target_length(data, target_length)
                    return data
                else:
                    return None
        except:
            return None

    def _adjust_to_target_length(self, data, target_length):
        current_length = data.shape[0]

        if current_length == target_length:
            return data
        elif current_length > target_length:
            excess = current_length - target_length
            start_trim = excess // 2
            end_trim = start_trim + target_length
            return data[start_trim:end_trim]
        else:
            padding_needed = target_length - current_length
            padding_before = padding_needed // 2
            padding_after = padding_needed - padding_before
            return np.pad(data, ((padding_before, padding_after), (0, 0)), 'constant', constant_values=0)

    def _load_patient_metadata(self):
        patients = []
        patient_files = list(self.patients_path.glob("patient_*.json"))

        for patient_file in sorted(patient_files):
            try:
                with open(patient_file, 'r') as f:
                    patient_data = json.load(f)
                patients.append(patient_data)
            except:
                continue

        df = pd.DataFrame(patients)
        print(f"Loaded {len(df)} patients")
        if len(df) > 0:
            print(f"Conditions: {df['condition'].value_counts().to_dict()}")
        return df

    def _load_questionnaire_data(self):
        questionnaire_data = {}
        quest_files = list(self.questionnaire_path.glob("questionnaire_response_*.json"))

        successful = 0
        for quest_file in quest_files:
            try:
                patient_id = quest_file.stem.split('_')[-1]
                with open(quest_file, 'r') as f:
                    data = json.load(f)
                features = self._extract_questionnaire_features(data)
                if len(features) >= 10:
                    questionnaire_data[patient_id] = features
                    successful += 1
            except:
                continue

        print(f"Processed {successful} questionnaires")
        return questionnaire_data

    def _extract_questionnaire_features(self, questionnaire_data):
        features = {}
        if 'item' not in questionnaire_data:
            return features

        for item in questionnaire_data['item']:
            if 'link_id' not in item or 'answer' not in item:
                continue

            link_id = item['link_id']
            answer = item['answer']

            if isinstance(answer, bool):
                features[f'q_{link_id}'] = 1 if answer else 0
            elif isinstance(answer, (int, float)):
                features[f'q_{link_id}'] = float(answer)
            elif isinstance(answer, str):
                mapping = {
                    'never': 0, 'no': 0, 'false': 0, 'not at all': 0,
                    'rarely': 1, 'slight': 1, 'mild': 1,
                    'sometimes': 2, 'moderate': 2, 'occasionally': 2,
                    'often': 3, 'severe': 3, 'frequently': 3,
                    'always': 4, 'very severe': 4, 'extreme': 4
                }
                answer_clean = answer.lower().strip()
                features[f'q_{link_id}'] = mapping.get(answer_clean, abs(hash(answer_clean)) % 5)
            else:
                features[f'q_{link_id}'] = 0

        return features

# Initialize pipeline
data_pipeline = CleanDataPipeline(
    movement_dir=MOVEMENT_DIR,
    patients_dir=PATIENTS_DIR,
    questionnaire_dir=QUESTIONNAIRE_DIR
)

Found 469 .bin files
Dimensions: 976 × 132
Loaded 469 patients
Conditions: {"Parkinson's": 276, 'Healthy': 79, 'Other Movement Disorders': 60, 'Essential Tremor': 28, 'Atypical Parkinsonism': 15, 'Multiple Sclerosis': 11}
Processed 469 questionnaires


### Data Augmentation

In [7]:
class OnTheFlyAugmentation:
    def __init__(self):
        pass

    def apply_axis_rotation(self, data, max_angle=15):
        augmented_data = data.copy()
        for start_idx in range(0, data.shape[1], 6):
            if start_idx + 5 < data.shape[1]:
                angle = np.random.uniform(-max_angle, max_angle)
                rotation_matrix = self._get_rotation_matrix_3d(angle)

                # Rotate accelerometer data
                acc_data = data[:, start_idx:start_idx+3]
                rotated_acc = (rotation_matrix @ acc_data.T).T
                augmented_data[:, start_idx:start_idx+3] = rotated_acc

                # Rotate gyroscope data
                gyro_data = data[:, start_idx+3:start_idx+6]
                rotated_gyro = (rotation_matrix @ gyro_data.T).T
                augmented_data[:, start_idx+3:start_idx+6] = rotated_gyro

        return augmented_data

    def _get_rotation_matrix_3d(self, angle_deg):
        angle_rad = np.radians(angle_deg)
        axis = np.random.choice(['x', 'y', 'z'])
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)

        if axis == 'x':
            return np.array([[1, 0, 0], [0, cos_a, -sin_a], [0, sin_a, cos_a]])
        elif axis == 'y':
            return np.array([[cos_a, 0, sin_a], [0, 1, 0], [-sin_a, 0, cos_a]])
        else:
            return np.array([[cos_a, -sin_a, 0], [sin_a, cos_a, 0], [0, 0, 1]])

    def apply_time_warping(self, data, sigma=0.2, knot=4):
        orig_steps = np.arange(data.shape[0])
        random_warps = np.random.normal(loc=1.0, scale=sigma, size=(knot+2,))
        random_warps = np.abs(random_warps)

        time_warp = np.cumsum(random_warps)
        time_warp = time_warp / time_warp[-1] * (data.shape[0] - 1)
        warp_steps = np.linspace(0, data.shape[0] - 1, num=knot+2)

        warped_data = np.zeros_like(data)
        for i in range(data.shape[1]):
            try:
                warped_indices = np.interp(orig_steps, warp_steps, time_warp)
                warped_indices = np.clip(warped_indices, 0, data.shape[0] - 1)
                warped_data[:, i] = np.interp(warped_indices, orig_steps, data[:, i])
            except:
                warped_data[:, i] = data[:, i]

        return warped_data

    def apply_random_augmentation(self, data):
        augmented = data.copy()

        if np.random.random() < 0.6:
            augmented = self.apply_axis_rotation(augmented)

        if np.random.random() < 0.4:
            augmented = self.apply_time_warping(augmented)

        return augmented

augmentation = OnTheFlyAugmentation()

### Dataset Creation

In [8]:
def _to_quest_vector(quest_features: dict, size: int = 30) -> np.ndarray:
    """
    Convert a dict of questionnaire features -> fixed-length vector.
    Pads with zeros if fewer than `size` entries. Truncates otherwise.
    """
    v = np.zeros(size, dtype=np.float32)
    if not quest_features:
        return v
    items = list(quest_features.items())[:size]
    for i, (_, val) in enumerate(items):
        try:
            v[i] = float(val)
        except Exception:
            v[i] = 0.0
    return v

class CleanDatasetCreator:
    def __init__(self, data_pipeline):
        self.data_pipeline = data_pipeline

    def create_base_dataset(self, task_type='pd_vs_hc', modality='combined'):
        print(f"Creating {task_type} dataset ({modality})")

        if task_type == 'pd_vs_hc':
            valid_conditions = ["Parkinson's", "Healthy"]
            label_map = {"Healthy": 0, "Parkinson's": 1}
        elif task_type == 'pd_vs_dd':
            valid_conditions = ["Parkinson's", "Other Movement Disorders",
                              "Essential Tremor", "Atypical Parkinsonism", "Multiple Sclerosis"]
            label_map = {"Parkinson's": 1}
            for cond in valid_conditions[1:]:
                label_map[cond] = 0
        else:
            raise ValueError("Only classification tasks supported!")

        filtered_patients = self.data_pipeline.patient_metadata[
            self.data_pipeline.patient_metadata['condition'].isin(valid_conditions)
        ]

        X_movement = []
        X_questionnaire = []
        y_labels = []
        patient_ids = []
        successful_loads = 0

        for idx, row in tqdm(filtered_patients.iterrows(), total=len(filtered_patients)):
          patient_id = int(row['id'])
          pid_str    = f"{patient_id:03d}"
          condition  = row['condition']

          movement_data  = self.data_pipeline.load_bin_file_raw(patient_id)
          quest_features = self.data_pipeline.questionnaire_data.get(pid_str, {})

          has_mov   = movement_data is not None
          has_quest = len(quest_features) >= 10

          if modality == 'movement_only':
              if has_mov:
                  X_movement.append(movement_data.astype(np.float32))
                  X_questionnaire.append(np.zeros(30, dtype=np.float32))
                  y_labels.append(int(label_map[condition]))
                  patient_ids.append(patient_id)
                  successful_loads += 1

          elif modality == 'questionnaire_only':
              if has_quest:
                  quest_vector = _to_quest_vector(quest_features, size=30)
                  X_movement.append(np.zeros((950, 132), dtype=np.float32))
                  X_questionnaire.append(quest_vector)
                  y_labels.append(int(label_map[condition]))
                  patient_ids.append(patient_id)
                  successful_loads += 1

          elif modality == 'combined':
              if has_mov and has_quest:
                  quest_vector = _to_quest_vector(quest_features, size=30)
                  X_movement.append(movement_data.astype(np.float32))
                  X_questionnaire.append(quest_vector)
                  y_labels.append(int(label_map[condition]))
                  patient_ids.append(patient_id)
                  successful_loads += 1

          else:
              raise ValueError("Unsupported modality")


        X_movement = np.array(X_movement)
        X_questionnaire = np.array(X_questionnaire)
        y_labels = np.array(y_labels)

        unique, counts = np.unique(y_labels, return_counts=True)
        if task_type == 'pd_vs_hc':
            class_names = ['HC', 'PD']
        else:
            class_names = ['DD', 'PD']

        class_dist = dict(zip(class_names, counts))
        print(f"Loaded {successful_loads} patients: {class_dist}")

        return {
            'X_movement': X_movement,
            'X_questionnaire': X_questionnaire,
            'y': y_labels,
            'patient_ids': patient_ids,
            'task_type': task_type,
            'modality': modality,
            'successful_loads': successful_loads
        }
# helper to slice 132->per-task 12ch
def _slice_task_12ch(x132: np.ndarray, task_idx: int) -> np.ndarray:
    """
    x132: [T,132] ordered as 11 tasks × (Left/Right × Acc/Gyro × X/Y/Z) -> 11 * 12 = 132
    task_idx: 0..10
    returns [T,12]
    """
    base = task_idx * 12
    return x132[:, base:base+12].astype(np.float32)

# segment-level dataset builder
def create_segment_dataset(data_pipeline, task_type='pd_vs_hc', modality='movement_only'):
    print(f"Creating SEGMENT dataset: {task_type} ({modality})")

    if task_type == 'pd_vs_hc':
        valid_conditions = ["Parkinson's", "Healthy"]
        label_map = {"Healthy": 0, "Parkinson's": 1}
    elif task_type == 'pd_vs_dd':
        valid_conditions = ["Parkinson's", "Other Movement Disorders",
                            "Essential Tremor", "Atypical Parkinsonism", "Multiple Sclerosis"]
        label_map = {"Parkinson's": 1}
        for cond in valid_conditions[1:]:
            label_map[cond] = 0
    else:
        raise ValueError("Only classification tasks supported!")

    filtered = data_pipeline.patient_metadata[
        data_pipeline.patient_metadata['condition'].isin(valid_conditions)
    ]

    X_movement, X_questionnaire, y, pids = [], [], [], []
    TASKS_N = 11  # physio dataset has 11 tasks → 11×12ch = 132

    for _, row in filtered.iterrows():
        sid = int(row['id']); pid_str = f"{sid:03d}"
        cond = row['condition']

        mov132 = data_pipeline.load_bin_file_raw(sid)        # [950,132] or None
        quest  = data_pipeline.questionnaire_data.get(pid_str, {})
        has_mov   = mov132 is not None
        has_quest = len(quest) >= 10

        # make a fixed 30-d questionnaire vector (or zeros)
        quest_vec = np.zeros(30, dtype=np.float32)
        if has_quest:
            q = _to_quest_vector(quest, size=30)
            quest_vec = q.astype(np.float32)

        # explode to segments
        for t in range(TASKS_N):
            # decide inclusion by modality
            if modality == 'movement_only' and has_mov:
                x12 = _slice_task_12ch(mov132, t)  # [950,12]
                X_movement.append(x12)
                X_questionnaire.append(np.zeros(30, dtype=np.float32))
                y.append(int(label_map[cond]))
                pids.append(sid)

            elif modality == 'questionnaire_only' and has_quest:
                # no movement, but keep 12ch placeholder for consistency
                X_movement.append(np.zeros((950, 12), dtype=np.float32))
                X_questionnaire.append(quest_vec)
                y.append(int(label_map[cond]))
                pids.append(sid)

            elif modality == 'combined' and has_mov and has_quest:
                x12 = _slice_task_12ch(mov132, t)
                X_movement.append(x12)
                X_questionnaire.append(quest_vec)
                y.append(int(label_map[cond]))
                pids.append(sid)

    X_movement      = np.asarray(X_movement, dtype=np.float32)       # [Nseg, 950, 12]
    X_questionnaire = np.asarray(X_questionnaire, dtype=np.float32)  # [Nseg, 30]
    y               = np.asarray(y, dtype=np.int64)

    print(f"[segment] Built {len(y)} samples "
          f"({np.sum(y==0)} neg / {np.sum(y==1)} pos); "
          f"movement shape={X_movement.shape}, quest shape={X_questionnaire.shape}")

    return {
        'X_movement': X_movement,
        'X_questionnaire': X_questionnaire,
        'y': y,
        'patient_ids': pids,
        'task_type': task_type,
        'modality': modality,
        'split_unit': 'segment'
    }
dataset_creator = CleanDatasetCreator(data_pipeline)
# build both SUBJECT and SEGMENT datasets
datasets_subject = {}
datasets_segment = {}

for task in ['pd_vs_hc', 'pd_vs_dd']:
    for modality in ['movement_only', 'questionnaire_only', 'combined']:
        key = f'{task}_{modality}'

        # SUBJECT version
        datasets_subject[key] = dataset_creator.create_base_dataset(task, modality)

        # SEGMENT version (12ch per segment)
        datasets_segment[key] = create_segment_dataset(data_pipeline, task, modality)

# Convenience mapping so we can pick by SPLIT_UNIT later
DATASET_POOLS = {
    "subject": datasets_subject,
    "segment": datasets_segment,
}
print(f"SUBJECT datasets: {list(datasets_subject.keys())}")
print(f"SEGMENT datasets: {list(datasets_segment.keys())}")

Creating pd_vs_hc dataset (movement_only)


100%|██████████| 355/355 [03:01<00:00,  1.96it/s]


Loaded 355 patients: {'HC': np.int64(79), 'PD': np.int64(276)}
Creating SEGMENT dataset: pd_vs_hc (movement_only)
[segment] Built 3905 samples (869 neg / 3036 pos); movement shape=(3905, 950, 12), quest shape=(3905, 30)
Creating pd_vs_hc dataset (questionnaire_only)


100%|██████████| 355/355 [00:00<00:00, 401.93it/s]


Loaded 355 patients: {'HC': np.int64(79), 'PD': np.int64(276)}
Creating SEGMENT dataset: pd_vs_hc (questionnaire_only)
[segment] Built 3905 samples (869 neg / 3036 pos); movement shape=(3905, 950, 12), quest shape=(3905, 30)
Creating pd_vs_hc dataset (combined)


...

Loaded 390 patients: {'DD': np.int64(114), 'PD': np.int64(276)}
Creating SEGMENT dataset: pd_vs_dd (movement_only)
[segment] Built 4290 samples (1254 neg / 3036 pos); movement shape=(4290, 950, 12), quest shape=(4290, 30)
Creating pd_vs_dd dataset (questionnaire_only)


100%|██████████| 390/390 [00:01<00:00, 379.66it/s]


Loaded 390 patients: {'DD': np.int64(114), 'PD': np.int64(276)}
Creating SEGMENT dataset: pd_vs_dd (questionnaire_only)
[segment] Built 4290 samples (1254 neg / 3036 pos); movement shape=(4290, 950, 12), quest shape=(4290, 30)
Creating pd_vs_dd dataset (combined)


100%|██████████| 390/390 [00:00<00:00, 404.66it/s]


Loaded 390 patients: {'DD': np.int64(114), 'PD': np.int64(276)}
Creating SEGMENT dataset: pd_vs_dd (combined)
[segment] Built 4290 samples (1254 neg / 3036 pos); movement shape=(4290, 950, 12), quest shape=(4290, 30)
SUBJECT datasets: ['pd_vs_hc_movement_only', 'pd_vs_hc_questionnaire_only', 'pd_vs_hc_combined', 'pd_vs_dd_movement_only', 'pd_vs_dd_questionnaire_only', 'pd_vs_dd_combined']
SEGMENT datasets: ['pd_vs_hc_movement_only', 'pd_vs_hc_questionnaire_only', 'pd_vs_hc_combined', 'pd_vs_dd_movement_only', 'pd_vs_dd_questionnaire_only', 'pd_vs_dd_combined']


### Model Definitions (Universal)

In [9]:
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.mlp = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction_ratio, channels, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size, channels, _ = x.size()
        y = self.avg_pool(x).view(batch_size, channels)
        y = self.mlp(y)
        y = self.sigmoid(y).view(batch_size, channels, 1)
        return y

class CA_LSTM_FCN(nn.Module):
    """
    FCN(8/5/3)+ChannelAttention+GAP  ||  LSTM(last state)  -> concat -> head
    - num_classes=2 -> CrossEntropy (baseline)
    Handles all modalities gracefully: movement_only / questionnaire_only / combined
    """
    def __init__(self, input_channels=12, sequence_length=950, num_classes=1,
                 questionnaire_features=30, lstm_hidden_size=128,
                 fcn_filters=(128,256,128), dropout=0.2, modality='movement_only'):
        super().__init__()
        self.modality = modality
        self.use_move  = modality in ('movement_only','combined')
        self.use_quest = modality in ('questionnaire_only','combined')

        # --- Movement branch (optional) ---
        movement_feat_dim = 0
        if self.use_move:
            C = input_channels
            self.fcn = nn.Sequential(
                nn.Conv1d(C, fcn_filters[0], kernel_size=8, padding=4),
                nn.BatchNorm1d(fcn_filters[0]), nn.ReLU(),
                nn.Conv1d(fcn_filters[0], fcn_filters[1], kernel_size=5, padding=2),
                nn.BatchNorm1d(fcn_filters[1]), nn.ReLU(),
                nn.Conv1d(fcn_filters[1], fcn_filters[2], kernel_size=3, padding=1),
                nn.BatchNorm1d(fcn_filters[2]), nn.ReLU(),
            )
            self.cam = ChannelAttention(fcn_filters[-1])
            self.gap = nn.AdaptiveAvgPool1d(1)

            self.lstm = nn.LSTM(
                input_size=C,
                hidden_size=lstm_hidden_size,
                num_layers=1,
                batch_first=True,
                dropout=0.0,
                bidirectional=False
            )
            self.dp_lstm = nn.Dropout(dropout)
            movement_feat_dim = fcn_filters[-1] + lstm_hidden_size
        else:
            # placeholders so attributes exist even if unused
            self.fcn = None
            self.cam = None
            self.gap = None
            self.lstm = None
            self.dp_lstm = None

        # --- Questionnaire branch (optional) ---
        self.qhead = None
        q_dim = 0
        if self.use_quest:
            self.qhead = nn.Sequential(
                nn.Linear(questionnaire_features, 64), nn.ReLU(), nn.Dropout(dropout),
                nn.Linear(64, 128), nn.ReLU(), nn.Dropout(dropout)
            )
            q_dim = 128

        # --- Fusion head ---
        fused_dim = movement_feat_dim + q_dim
        self.head = nn.Sequential(
            nn.Linear(fused_dim, 128), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

        # --- Weight initialization ---
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None: nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None: nn.init.constant_(m.bias, 0)

    def forward(self, movement_data, clinical_data):
        feats = []

        if self.use_move and movement_data is not None and movement_data.numel() > 0:
            x = movement_data
            f = self.fcn(x.transpose(1, 2))
            f = f * self.cam(f)
            f = self.gap(f).squeeze(-1)
            h, _ = self.lstm(x)
            m = self.dp_lstm(h[:, -1, :])
            feats.extend([f, m])

        if self.use_quest and clinical_data is not None and clinical_data.numel() > 0:
            q = self.qhead(clinical_data)
            feats.append(q)

        if not feats:
            raise RuntimeError("CA_LSTM_FCN received no valid inputs (check modality configuration).")

        z = torch.cat(feats, dim=1)
        return self.head(z)

In [10]:
class TimeSeriesTransformerConfig(PretrainedConfig):
    model_type = "time_series_transformer"

    def __init__(self, sequence_length=950, input_channels=132, num_classes=2,
                 questionnaire_features=30, d_model=256, nhead=8, num_layers=4,
                 dropout=0.1, modality='combined', **kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length
        self.input_channels = input_channels
        self.num_classes = num_classes
        self.questionnaire_features = questionnaire_features
        self.d_model = d_model
        self.nhead = nhead
        self.num_layers = num_layers
        self.dropout = dropout
        self.modality = modality


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1024):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # [1, max_len, d_model]

    def forward(self, x):
        # x: [B, T, d]
        return x + self.pe[:, :x.size(1)]


class TimeSeriesTransformer(PreTrainedModel):
    config_class = TimeSeriesTransformerConfig

    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.use_movement = config.modality in ['movement_only', 'combined']
        self.use_questionnaire = config.modality in ['questionnaire_only', 'combined']

        # Projections
        if self.use_movement:
            self.movement_projection = nn.Linear(config.input_channels, config.d_model)
        if self.use_questionnaire:
            self.questionnaire_projection = nn.Linear(config.questionnaire_features, config.d_model)
            if self.use_movement:
                # Early fusion: per-timestep concat(movement, questionnaire) -> project back to d_model
                self.early_fusion = nn.Sequential(
                    nn.Linear(config.d_model * 2, config.d_model),
                    nn.ReLU(),
                    nn.Dropout(config.dropout)
                )

        # +1 for CLS token (max sequence length we need to encode)
        max_seq_len = config.sequence_length + 1
        self.pos_encoding = PositionalEncoding(config.d_model, max_seq_len)
        self.input_layer_norm = nn.LayerNorm(config.d_model)

        # CLS token
        self.cls = nn.Parameter(torch.randn(1, 1, config.d_model))

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config.d_model,
            nhead=config.nhead,
            dim_feedforward=config.d_model * 4,
            dropout=config.dropout,
            activation='relu',
            batch_first=True,
            norm_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=config.num_layers,
            norm=nn.LayerNorm(config.d_model)
        )

        # Classifier on CLS
        self.classifier = nn.Sequential(
            nn.LayerNorm(config.d_model),
            nn.Dropout(config.dropout),
            nn.Linear(config.d_model, config.d_model // 2),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.d_model // 2, config.num_classes)
        )

        self.init_weights()

    def forward(self, movement_data, clinical_data=None, key_padding_mask=None):
        """
        movement_data: [B, T, C]
        clinical_data: [B, F] or None
        key_padding_mask (optional): [B, T] with True marking PAD positions (if you use variable T)
        """
        if not (self.use_movement or self.use_questionnaire):
            raise ValueError("Model modality must be one of ['movement_only','questionnaire_only','combined'].")

        sequences = []

        if self.use_movement and self.use_questionnaire:
            # Project movement to d_model
            mov = self.movement_projection(movement_data)                 # [B, T, d]
            # Project questionnaire to d_model and tile across time
            que = self.questionnaire_projection(clinical_data)            # [B, d]
            que_tiled = que.unsqueeze(1).expand(-1, mov.size(1), -1)      # [B, T, d]
            fused = torch.cat([mov, que_tiled], dim=-1)                   # [B, T, 2d]
            x = self.early_fusion(fused)                                  # [B, T, d]
            sequences.append(x)

        elif self.use_movement:
            x = self.movement_projection(movement_data)                   # [B, T, d]
            sequences.append(x)

        elif self.use_questionnaire:
            que = self.questionnaire_projection(clinical_data)            # [B, d]
            x = que.unsqueeze(1)                                          # [B, 1, d]
            sequences.append(x)

        x = torch.cat(sequences, dim=1) if len(sequences) > 1 else sequences[0]  # [B, T_or_1, d]

        # Prepend CLS
        B = x.size(0)
        cls_tok = self.cls.expand(B, 1, -1)                               # [B, 1, d]
        x = torch.cat([cls_tok, x], dim=1)                                # [B, 1+T, d]

        # Optional key padding mask: prepend False for CLS
        kpm = None
        if key_padding_mask is not None:
            if key_padding_mask.dim() != 2 or key_padding_mask.size(0) != B:
                raise ValueError("key_padding_mask must be [B, T]")
            kpm = torch.cat([torch.zeros(B, 1, dtype=torch.bool, device=key_padding_mask.device),
                             key_padding_mask], dim=1)                     # [B, 1+T]

        # Encode (LN -> PE -> Transformer)
        x = self.input_layer_norm(x)
        x = self.pos_encoding(x)  # add sinusoidal PE
        h = self.transformer_encoder(x, src_key_padding_mask=kpm)


        # CLS pooling
        z = h[:, 0, :]                                                    # [B, d]
        logits = self.classifier(z)                                       # [B, num_classes]
        return logits

### Model Hyperparameter Search Space

In [11]:
# hyperparameter search space
model_hyperparameters = {
    'CA-LSTM-FCN': {
        'lstm_hidden_size': [64, 128],
        'fcn_filters': [[64, 128, 64], [128, 256, 128]],
        'dropout': [0.1, 0.2]
    },
    'Transformer': {
        'd_model': [128, 256],
        'nhead': [4, 8],
        'num_layers': [2, 3],
        'dropout': [0.1, 0.2]
    }
}

print("Finalized hyperparameters for tuning:")
for model_name, params in model_hyperparameters.items():
    print(f"- {model_name}:")
    for hp, values in params.items():
        print(f"  - {hp}: {values}")

Finalized hyperparameters for tuning:
- CA-LSTM-FCN:
  - lstm_hidden_size: [64, 128]
  - fcn_filters: [[64, 128, 64], [128, 256, 128]]
  - dropout: [0.1, 0.2]
- Transformer:
  - d_model: [128, 256]
  - nhead: [4, 8]
  - num_layers: [2, 3]
  - dropout: [0.1, 0.2]


### Dataset and Trainer Definitions

In [12]:
# ---- Dataset ----
class TrainingDataset(Dataset):
    def __init__(self, movement_data, clinical_data, labels, indices,
                 movement_scaler=None, clinical_scaler=None,
                 augmentation=None, is_training=False, modality='combined'):

        self.movement_data = movement_data[indices]
        self.clinical_data = clinical_data[indices]
        self.labels = labels[indices]
        self.movement_scaler = movement_scaler
        self.clinical_scaler = clinical_scaler
        self.augmentation = augmentation
        self.is_training = is_training
        self.modality = modality
        self.paper_parity = False  # default; set True for parity runs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        movement = self.movement_data[idx].copy()
        clinical = self.clinical_data[idx].copy()
        label = int(self.labels[idx])

        # --- 1. Scale movement ---
        if self.movement_scaler is not None and self.modality in ['movement_only', 'combined']:
            T, C = movement.shape[-2], movement.shape[-1]
            movement = self.movement_scaler.transform(movement.reshape(-1, C)).reshape(T, C)

        # --- 2. Paper-parity additive Gaussian noise (only in training) ---
        if self.is_training and self.paper_parity and self.modality in ['movement_only','combined']:
            movement += np.random.normal(0.0, 0.01, size=movement.shape).astype(np.float32)

        # --- 3. Scale clinical features ---
        if self.clinical_scaler is not None and self.modality in ['questionnaire_only', 'combined']:
            clinical = self.clinical_scaler.transform(clinical.reshape(1, -1)).ravel()

        # --- 4. Baseline augmentation (disabled if paper_parity=True) ---
        if self.is_training and not self.paper_parity and self.augmentation is not None \
          and self.modality in ['movement_only','combined']:

            model_name = getattr(self, 'current_model_name', None)
            if label == 0:  # augment controls only
                if model_name == 'CA-LSTM-FCN':
                    # stronger augmentations
                    if np.random.rand() < 0.6:
                        movement = self.augmentation.apply_axis_rotation(movement, max_angle=12)
                    if np.random.rand() < 0.4:
                        movement = self.augmentation.apply_time_warping(movement, sigma=0.20, knot=4)
                    if np.random.rand() < 0.2:
                        movement += np.random.normal(0, 0.01, movement.shape)
                else:
                    # default weaker augmentations
                    if np.random.rand() < 0.6:
                        movement = self.augmentation.apply_axis_rotation(movement, max_angle=5)
                    if np.random.rand() < 0.4:
                        movement = self.augmentation.apply_time_warping(movement, sigma=0.10, knot=4)

        return (
            torch.tensor(movement, dtype=torch.float32),
            torch.tensor(clinical, dtype=torch.float32),
            torch.tensor(label, dtype=torch.long)
        )

# ---- Trainer ----
class Trainer:
    def __init__(self, device='cuda'):
        self.device = device
        self.augmentation = OnTheFlyAugmentation()
        self.paper_parity = False

    def train_single_fold(self, model, train_dataset, val_dataset, max_epochs, fold_num):
        model = model.to(self.device)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)

        # === 1. Criterion selection ===
        if isinstance(model, CA_LSTM_FCN) and model.head[-1].out_features == 1:
            criterion = nn.BCEWithLogitsLoss()
        else:
            criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

        # === 2. Optimizer & scheduler ===
        optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
        if self.paper_parity:
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=8)
        else:
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)

        # === Best tracking / early stopping ===
        # Monitor = F1 in parity mode (paper), else BA (previous baseline).
        best_monitor = -1.0
        patience = 35
        patience_counter = 0

        best_epoch = -1
        best_at_monitor = {  # metrics at the epoch where the monitor is best
            'acc': -1.0, 'ba': -1.0, 'prec': -1.0, 'rec': -1.0, 'f1': -1.0
        }
        # true maxima across epochs
        best_ba_any_epoch = -1.0
        best_f1_any_epoch = -1.0

        # keep last-epoch metrics for reference
        acc = ba = prec = rec = f1 = 0.0

        for epoch in range(max_epochs):
            # ---- Train ----
            model.train()
            train_loss_sum = 0.0
            train_batches = 0
            for movement_data, clinical_data, labels in train_loader:
                movement_data = movement_data.to(self.device)
                clinical_data = clinical_data.to(self.device)
                labels        = labels.to(self.device)

                optimizer.zero_grad()
                outputs = model(movement_data, clinical_data)

                if isinstance(criterion, nn.BCEWithLogitsLoss):
                    labels_float = labels.float().unsqueeze(1)
                    loss = criterion(outputs, labels_float)
                else:
                    loss = criterion(outputs, labels)

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                train_loss_sum += float(loss.detach().cpu().item())
                train_batches += 1

            # ---- Validate ----
            model.eval()
            val_predictions, val_targets = [], []
            val_loss_sum = 0.0
            val_batches = 0
            with torch.no_grad():
                for movement_data, clinical_data, labels in val_loader:
                    movement_data = movement_data.to(self.device)
                    clinical_data = clinical_data.to(self.device)
                    labels        = labels.to(self.device)

                    outputs = model(movement_data, clinical_data)
                    if isinstance(criterion, nn.BCEWithLogitsLoss):
                        preds = (torch.sigmoid(outputs) > 0.5).long().cpu().numpy().flatten()
                        vloss = criterion(outputs, labels.float().unsqueeze(1))
                    else:
                        preds = torch.argmax(outputs, dim=1).cpu().numpy()
                        vloss = criterion(outputs, labels)

                    val_predictions.extend(preds)
                    val_targets.extend(labels.cpu().numpy())
                    val_loss_sum += float(vloss.detach().cpu().item())
                    val_batches += 1
            # ---- Metrics ----
            from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
            acc  = accuracy_score(val_targets, val_predictions)
            ba   = balanced_accuracy_score(val_targets, val_predictions)
            prec = precision_score(val_targets, val_predictions, zero_division=0)
            rec  = recall_score(val_targets, val_predictions, zero_division=0)
            f1   = f1_score(val_targets, val_predictions, zero_division=0)

            # update "true" bests (independent of monitor)
            if ba > best_ba_any_epoch:
                best_ba_any_epoch = ba
            if f1 > best_f1_any_epoch:
                best_f1_any_epoch = f1

            # choose monitor (for early stopping + plateau scheduler)
            monitor = f1 if self.paper_parity else ba
            if self.paper_parity:
                scheduler.step(monitor)   # ReduceLROnPlateau needs a metric
            else:
                scheduler.step()          # cosine annealing ignores metric

            # keep metrics at the best monitor epoch
            if monitor > best_monitor:
                best_monitor = monitor
                best_epoch = epoch + 1
                patience_counter = 0
                best_at_monitor.update({'acc': acc, 'ba': ba, 'prec': prec, 'rec': rec, 'f1': f1})
            else:
                patience_counter += 1

            if epoch % 20 == 0 or epoch < 3:
                tl = train_loss_sum / max(1, train_batches)
                vl = val_loss_sum   / max(1, val_batches)
                print(f"    Fold {fold_num}, Epoch {epoch+1:3d}: "
                      f"TrainLoss={tl:.4f} | ValLoss={vl:.4f} | "
                      f"Acc={acc:.3f} | BA={ba:.3f} | F1={f1:.3f} | Prec={prec:.3f} | Rec={rec:.3f}")

            if patience_counter >= patience:
                print(f"    Early stopping at epoch {epoch+1}")
                break

        return {
            'best_epoch': best_epoch,
            'best_acc': best_at_monitor['acc'],
            'best_ba': best_at_monitor['ba'],
            'best_prec': best_at_monitor['prec'],
            'best_rec': best_at_monitor['rec'],
            'best_f1': best_at_monitor['f1'],

            'best_ba_any_epoch': best_ba_any_epoch,
            'best_f1_any_epoch': best_f1_any_epoch,

            # last-epoch metrics
            'last_acc': acc, 'last_ba': ba, 'last_prec': prec, 'last_rec': rec, 'last_f1': f1
        }



    def comprehensive_evaluation(self, model_class, model_params_space, dataset, max_epochs=60):
        task_type = dataset['task_type']
        modality  = dataset['modality']
        model_name = model_class.__name__
        print(f"\nEvaluating {model_name} on {task_type} ({modality})")

        X_mov_train_val   = dataset['X_movement']
        X_quest_train_val = dataset['X_questionnaire']
        y_train_val       = dataset['y']

        patient_ids_train_val = dataset.get('patient_ids', None)

        print("  Using full dataset for CV.")

        # --- Hyperparameter Tuning Loop ---
        best_cv_score = -1.0
        best_hyperparameters = None
        best_fold_scores = None
        best_model_state = None
        best_movement_scaler_saved = None
        best_clinical_scaler_saved = None

        # grid
        param_names  = list(model_params_space.keys())
        param_values = list(model_params_space.values())
        hp_grid = list(itertools.product(*param_values))
        print(f"  Starting Grid Search with {len(hp_grid)} combinations...")

        for hp_tuple in hp_grid:
            current_params = dict(zip(param_names, hp_tuple))
            print(f"  Evaluating HPs: {current_params}")

            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            cv_splits = list(skf.split(X_mov_train_val, y_train_val))

            fold_scores = []
            fold_model_states = []
            fold_movement_scalers = []
            fold_clinical_scalers = []

            for fold_idx, (train_idx, val_idx) in enumerate(cv_splits):
                # fit scalers on train only
                movement_scaler = None
                clinical_scaler = None
                if modality in ['movement_only', 'combined']:
                    movement_scaler = StandardScaler()
                    Xtr_mov = X_mov_train_val[train_idx]
                    movement_scaler.fit(Xtr_mov.reshape(-1, Xtr_mov.shape[-1]))
                if modality in ['questionnaire_only', 'combined']:
                    clinical_scaler = StandardScaler()
                    clinical_scaler.fit(X_quest_train_val[train_idx])

                # datasets
                train_dataset = TrainingDataset(
                    X_mov_train_val, X_quest_train_val, y_train_val,
                    train_idx, movement_scaler, clinical_scaler,
                    self.augmentation, is_training=True, modality=modality
                )
                val_dataset = TrainingDataset(
                    X_mov_train_val, X_quest_train_val, y_train_val,
                    val_idx, movement_scaler, clinical_scaler,
                    None, is_training=False, modality=modality
                )

                # instantiate model
                if model_class == TimeSeriesTransformer:
                    config = TimeSeriesTransformerConfig(modality=modality, **current_params)
                    model = TimeSeriesTransformer(config)
                else:
                    params_for_instantiation = {**current_params, 'modality': modality}
                    model = model_class(**params_for_instantiation)

                # train fold
                fold_score = self.train_single_fold(
                    model, train_dataset, val_dataset, max_epochs, fold_idx+1
                )
                fold_scores.append(fold_score)

                fold_model_states.append(model.state_dict())
                fold_movement_scalers.append(movement_scaler)
                fold_clinical_scalers.append(clinical_scaler)

                # cleanup
                del model
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                else:
                    torch.cuda.empty_cache()

            # per-hp stats
            mean_cv_score = float(np.mean([fs['best_ba'] for fs in fold_scores]))
            std_cv_score  = float(np.std(fold_scores))
            print(f"  HPs {current_params}: Mean CV Result = {mean_cv_score:.4f} ± {std_cv_score:.4f}")

            # best tracking
            if mean_cv_score > best_cv_score:
                best_cv_score = mean_cv_score
                best_hyperparameters = current_params
                best_fold_scores = list(fold_scores)

                best_fold_index_in_hp_run = int(np.argmax(fold_scores))
                best_model_state = fold_model_states[best_fold_index_in_hp_run]
                best_movement_scaler_saved = fold_movement_scalers[best_fold_index_in_hp_run]
                best_clinical_scaler_saved = fold_clinical_scalers[best_fold_index_in_hp_run]

        print(f"\n  Best HPs found: {best_hyperparameters}")
        print(f"  Best Mean CV Score: {best_cv_score:.4f}")
        best_std = float(np.std(best_fold_scores)) if best_fold_scores else None
        print(f"  Std of Best HP CV: {best_std if best_std is not None else 'N/A'}")

        return {
            'best_hyperparameters': best_hyperparameters,
            'mean_cv_score': best_cv_score,
            'std_cv_score': best_std,
            'fold_scores': best_fold_scores,
            'best_model_state': best_model_state,
            'best_movement_scaler': best_movement_scaler_saved,
            'best_clinical_scaler': best_clinical_scaler_saved,
        }

trainer = Trainer(device=device)


### Model Configuration

In [14]:
# Define models and their hyperparameter search spaces
models_config = {
    'CA-LSTM-FCN': {
        'class': CA_LSTM_FCN,
        'params_space': model_hyperparameters['CA-LSTM-FCN']
    },
    'Transformer': {
        'class': TimeSeriesTransformer,
        'params_space': model_hyperparameters['Transformer']
    }
}

print(f"Models: {list(models_config.keys())}")
print(f"SUBJECT datasets: {list(datasets_subject.keys())}")
print(f"SEGMENT datasets: {list(datasets_segment.keys())}")


Models: ['CA-LSTM-FCN', 'Transformer']
SUBJECT datasets: ['pd_vs_hc_movement_only', 'pd_vs_hc_questionnaire_only', 'pd_vs_hc_combined', 'pd_vs_dd_movement_only', 'pd_vs_dd_questionnaire_only', 'pd_vs_dd_combined']
SEGMENT datasets: ['pd_vs_hc_movement_only', 'pd_vs_hc_questionnaire_only', 'pd_vs_hc_combined', 'pd_vs_dd_movement_only', 'pd_vs_dd_questionnaire_only', 'pd_vs_dd_combined']


### Framework 1 Hyperparameter Tuning Run (no need to run this cell)


In [ ]:
# # ==== Hyperparameter tuning / evaluation ====

# # Choose which split unit you want to tune on
# SPLIT_UNIT = "subject"   # or "segment" (use subject as default, segment is an experimentary case for data leakage demonstration)

# # Optionally restrict which datasets/models to run (leave as None to run all)
# TARGET_DATASET_NAMES = None
# TARGET_MODEL_NAMES   = None

# # Pick the correct pool
# if SPLIT_UNIT not in DATASET_POOLS:
#     raise ValueError("SPLIT_UNIT must be 'subject' or 'segment'")
# dataset_pool = DATASET_POOLS[SPLIT_UNIT]

# # Resolve datasets to process
# datasets_to_process = list(dataset_pool.items())
# if TARGET_DATASET_NAMES:
#     datasets_to_process = [(name, data) for name, data in datasets_to_process
#                            if name in set(TARGET_DATASET_NAMES)]
#     if not datasets_to_process:
#         print(f"Warning: no datasets matched TARGET_DATASET_NAMES: {TARGET_DATASET_NAMES}")

# # Resolve models to process (keep order of TARGET_MODEL_NAMES if given)
# models_to_process = list(models_config.items())
# if TARGET_MODEL_NAMES:
#     missing = [m for m in TARGET_MODEL_NAMES if m not in models_config]
#     if missing:
#         print(f"Warning: unknown models in TARGET_MODEL_NAMES: {missing}")
#     models_to_process = [(m, models_config[m]) for m in TARGET_MODEL_NAMES if m in models_config]

# print("Datasets to process:", [n for n, _ in datasets_to_process])
# print("Models to process:",   [n for n, _ in models_to_process])

# all_results = {}
# for dataset_name, dataset in datasets_to_process:
#     print(f"\nDATASET: {dataset_name.upper()}")
#     task_results = {}

#     for model_name, model_config in models_to_process:
#         print(f"--- {model_name} ---")
#         result = trainer.comprehensive_evaluation(
#             model_config['class'],
#             model_config['params_space'],
#             dataset,
#             max_epochs=60
#         )
#         task_results[model_name] = result

#     all_results[dataset_name] = task_results

# # ---- Summarize & save ----
# print("\nCREATING RESULTS SUMMARY")
# rows = []
# for dataset_name, task_results in all_results.items():
#     for model_name, result in task_results.items():
#         rows.append({
#             'Dataset': dataset_name,
#             'Model': model_name,
#             'Best_Hyperparameters': result['best_hyperparameters'],
#             'Mean_CV_Score': result['mean_cv_score'],
#             'Std_CV_Score': result['std_cv_score'],
#         })

# results_df = pd.DataFrame(rows)
# print(f"Results summary: {len(results_df)} entries")

# out_csv = RESULTS_DIR / 'evaluation_results_with_hp_tuning.csv'
# results_df.to_csv(out_csv, index=False)
# print(f"Results saved to: {out_csv}")

# print("\nTOP RESULTS (Sorted by Mean CV Score):")
# if not results_df.empty:
#     top_results = results_df.sort_values('Mean_CV_Score', ascending=False).head(10)
#     print(top_results[['Dataset', 'Model', 'Best_Hyperparameters', 'Mean_CV_Score', 'Std_CV_Score']].to_string(index=False))
# else:
#     print("No results to display.")

Models: ['CA-LSTM-FCN', 'Transformer', '1D-CNN', 'LSTM', 'MLP']
Datasets: ['pd_vs_hc_movement_only', 'pd_vs_hc_questionnaire_only', 'pd_vs_hc_combined', 'pd_vs_dd_movement_only', 'pd_vs_dd_questionnaire_only', 'pd_vs_dd_combined']
DATASET: PD_VS_HC_MOVEMENT_ONLY
--- CA-LSTM-FCN ---

Evaluating CA_LSTM_FCN on pd_vs_hc (movement_only)
  Using full dataset for CV.
  Starting Grid Search with 8 combinations...
  Evaluating HPs: {'lstm_hidden_size': 64, 'fcn_filters': [64, 128, 64], 'dropout': 0.1}
    Fold 1, Epoch   1: Bal Acc=0.5000
    Fold 1, Epoch   2: Bal Acc=0.5000


KeyboardInterrupt: 

### Framework 1 Evaluation with Best Hyperparameters


In [15]:
# === Framework 1 Evaluation with Best Hyperparameters ===

# --- Configuration ---
# Specify the datasets and models you want to evaluate with best HPs
# Set to None or empty list [] to evaluate all available datasets/models
TARGET_DATASET_NAMES = ["pd_vs_hc_movement_only","pd_vs_hc_questionnaire_only","pd_vs_hc_combined","pd_vs_dd_movement_only","pd_vs_dd_questionnaire_only","pd_vs_dd_combined"] # Example: ['pd_vs_hc_combined', 'pd_vs_dd_combined'] or None
TARGET_MODEL_NAMES   = ["Transformer","CA-LSTM-FCN"]      # Example: ['Transformer', 'CA-LSTM-FCN'] or None
SPLIT_UNIT = "subject"  # or "subject" --> Segment gives us biased results due to data leakage, use it with CALSTMFCN for paper comparison
PAPER_PARITY = False # flag to run with CALSTMFCN to compare with the Yu2024 electronics paper results for data leakage

# --- Choose dataset pool based on SPLIT_UNIT ---
if SPLIT_UNIT not in DATASET_POOLS:
    raise ValueError("SPLIT_UNIT must be 'subject' or 'segment'")
dataset_pool = DATASET_POOLS[SPLIT_UNIT]

if 'datasets_subject' not in globals() or 'datasets_segment' not in globals():
    raise ValueError("Dataset pools not found. Please run the data preparation cells that build datasets_subject and datasets_segment.")

# --- Load Best Hyperparameters Results ---
HP_CSV_PATH = RESULTS_DIR / 'evaluation_results_with_hp_tuning.csv'

try:
    hp_df_eval = pd.read_csv(HP_CSV_PATH)
    print(f"Loaded HP results from: {HP_CSV_PATH}")
except FileNotFoundError:
    print(f"Error: HP results CSV not found at {HP_CSV_PATH}. Cannot load best hyperparameters.")
    hp_df_eval = pd.DataFrame()


# --- Prepare Trainer ---
trainer_single_run = Trainer(device=device)

# --- Choose dataset pool based on SPLIT_UNIT ---
if SPLIT_UNIT == "subject":
    dataset_pool = datasets_subject
elif SPLIT_UNIT == "segment":
    dataset_pool = datasets_segment
else:
    raise ValueError("SPLIT_UNIT must be 'subject' or 'segment'")

# --- Determine which datasets and models to evaluate ---
datasets_to_process = dataset_pool.items()
if TARGET_DATASET_NAMES:
    datasets_to_process = [(name, data) for name, data in dataset_pool.items()
                           if name in TARGET_DATASET_NAMES]
    if not datasets_to_process:
        print(f"Warning: No datasets found matching TARGET_DATASET_NAMES: {TARGET_DATASET_NAMES}")

# Build models_to_process in the exact order of TARGET_MODEL_NAMES
if TARGET_MODEL_NAMES:
    missing = [m for m in TARGET_MODEL_NAMES if m not in models_config]
    if missing:
        print(f"Warning: These models are not in models_config and will be skipped: {missing}")
    models_to_process = [(name, models_config[name]) for name in TARGET_MODEL_NAMES if name in models_config]
else:
    models_to_process = list(models_config.items())

print("Models to process (in order):", [name for name, _ in models_to_process])

print("\nStarting evaluation with best hyperparameters...")

all_best_hp_results = []

# Loop through selected datasets
for dataset_name, dataset_to_evaluate in datasets_to_process:
    print(f"\n--- Evaluating on Dataset: {dataset_name.upper()} ---")

    X_mov_train_val   = dataset_to_evaluate['X_movement']
    X_quest_train_val = dataset_to_evaluate['X_questionnaire']
    y_train_val       = dataset_to_evaluate['y']
    modality          = dataset_to_evaluate['modality']
    task_type         = dataset_to_evaluate['task_type']


    # Loop through selected models
    for model_name, model_config in models_to_process:
        print(f"\n  Evaluating Model: {model_name}")

        # --- Load Best Hyperparameters for the Current Model and Dataset ---
        best_hps = {}
        if not hp_df_eval.empty:
            # Find the best HP row for this specific dataset and model
            expected_model_name_in_csv = f"{model_name}-EarlyFusion"
            if expected_model_name_in_csv not in hp_df_eval['Model'].unique():
                 expected_model_name_in_csv = model_name
                 if expected_model_name_in_csv not in hp_df_eval['Model'].unique():
                      print(f"  Warning: Model name '{model_name}' or '{model_name}-EarlyFusion' not found in CSV. Using default HPs.")
                      filtered_hps = pd.DataFrame()
                 else:
                      filtered_hps = hp_df_eval[
                         (hp_df_eval['Dataset'] == dataset_name) &
                         (hp_df_eval['Model'] == expected_model_name_in_csv)
                      ]
            else:
                 filtered_hps = hp_df_eval[
                    (hp_df_eval['Dataset'] == dataset_name) &
                    (hp_df_eval['Model'] == expected_model_name_in_csv)
                 ]


            if not filtered_hps.empty:
                best_row = filtered_hps.sort_values('Mean_CV_Score', ascending=False).iloc[0]
                import ast
                try:
                    best_hps = ast.literal_eval(best_row['Best_Hyperparameters'])
                    if not isinstance(best_hps, dict):
                         print(f"  Warning: Parsed best hyperparameters for {model_name} on {dataset_name} are not a dictionary. Using default HPs.")
                         best_hps = {}
                except (ValueError, SyntaxError) as e:
                    print(f"  Error parsing best hyperparameters string for {model_name} on {dataset_name}: {best_row['Best_Hyperparameters']}. Using default HPs. Error: {e}")
                    best_hps = {}
            else:
                pass


        # Use default HPs if not found or loaded incorrectly
        if not best_hps:
            print(f"  Using default hyperparameters for {model_name} on {dataset_name}.")
            if model_name == 'CA-LSTM-FCN':
                best_hps = {'lstm_hidden_size': 128, 'fcn_filters': [128, 256, 128], 'dropout': 0.2}
            elif model_name == 'Transformer':
                best_hps = {'d_model': 128, 'nhead': 4, 'num_layers': 3, 'dropout': 0.2}
            else:
                best_hps = {}
        print(f"  Loaded/Using Hyperparameters: {best_hps}")


        # --- Run 5-fold CV with Best HPs ---
        if SPLIT_UNIT == "segment":
            # Segment-level CV (allows leakage across the same subject by design)
            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            cv_splits = list(skf.split(X_mov_train_val, y_train_val))

        elif SPLIT_UNIT == "subject":
            # Subject-level CV: group by subject ID to prevent leakage
            patient_ids_all = np.array(dataset_to_evaluate['patient_ids'])
            unique_ids = np.unique(patient_ids_all)

            # One label per subject (majority vote across their samples)
            subject_labels = []
            for pid in unique_ids:
                mask = (patient_ids_all == pid)
                votes = y_train_val[mask]
                subject_labels.append(int(np.round(votes.mean())))  # binary case
            subject_labels = np.array(subject_labels)

            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            subj_splits = list(skf.split(np.zeros(len(unique_ids)), subject_labels))

            # Map subject folds back to sample indices
            cv_splits = []
            for tr_subj_idx, va_subj_idx in subj_splits:
                tr_subj_ids = unique_ids[tr_subj_idx]
                va_subj_ids = unique_ids[va_subj_idx]
                train_idx = np.where(np.isin(patient_ids_all, tr_subj_ids))[0]
                val_idx   = np.where(np.isin(patient_ids_all, va_subj_ids))[0]
                cv_splits.append((train_idx, val_idx))
        else:
            raise ValueError("SPLIT_UNIT must be 'subject' or 'segment'")

        fold_scores_acc, fold_scores_ba, fold_scores_prec, fold_scores_rec, fold_scores_f1 = [], [], [], [], []

        # Map model name string to class
        model_class_map = {
            'CA-LSTM-FCN': CA_LSTM_FCN,
            'Transformer': TimeSeriesTransformer
        }

        if model_name not in model_class_map:
             print(f"  Skipping unknown model name: {model_name}")
             continue

        model_class = model_class_map[model_name]


        for fold_idx, (train_idx, val_idx) in enumerate(cv_splits):
            # Default: use original arrays
            X_mov_tr,  X_quest_tr,  y_tr  = X_mov_train_val, X_quest_train_val, y_train_val
            tr_ids = train_idx

            # -------- PAPER PARITY: dataset-level cloning --------
            if PAPER_PARITY:
                y_tr_base = y_train_val[train_idx]
                is_neg = (y_tr_base == 0)
                neg_idx = train_idx[is_neg]
                pos_idx = train_idx[~is_neg]

                # Collect originals
                X_mov_list   = [X_mov_train_val[pos_idx],   X_mov_train_val[neg_idx]]
                X_quest_list = [X_quest_train_val[pos_idx], X_quest_train_val[neg_idx]]
                y_list       = [y_train_val[pos_idx],       y_train_val[neg_idx]]

                # Build parity-style clones
                n_extra = 2 if dataset_to_evaluate['task_type'] == 'pd_vs_hc' else 1  # HC×2 or DD×1
                for _ in range(n_extra):
                    Xn = X_mov_train_val[neg_idx].copy()
                    for i in range(Xn.shape[0]):
                        Xn[i] = augmentation.apply_axis_rotation(Xn[i], max_angle=12)
                        Xn[i] = augmentation.apply_time_warping(Xn[i], sigma=0.20, knot=4)
                    X_mov_list.append(Xn)
                    X_quest_list.append(X_quest_train_val[neg_idx])
                    y_list.append(y_train_val[neg_idx])

                # Concatenate augmented pool
                X_mov_tr   = np.concatenate(X_mov_list,   axis=0)
                X_quest_tr = np.concatenate(X_quest_list, axis=0)
                y_tr       = np.concatenate(y_list,       axis=0)
                tr_ids     = np.arange(len(y_tr))  # contiguous indices for dataset
            # --------------------------------------------------------------

            # ---- fit scalers on TRAIN ONLY (augmented pool if parity) ----
            movement_scaler = None
            clinical_scaler = None
            if modality in ['movement_only', 'combined']:
                movement_scaler = StandardScaler()
                Xm_fit = X_mov_tr
                if Xm_fit.ndim > 2:
                    movement_scaler.fit(Xm_fit.reshape(-1, Xm_fit.shape[-1]))
                else:
                    movement_scaler.fit(Xm_fit)
            if modality in ['questionnaire_only', 'combined']:
                clinical_scaler = StandardScaler()
                clinical_scaler.fit(X_quest_tr)

            # ---- datasets ----
            train_dataset = TrainingDataset(
                X_mov_tr, X_quest_tr, y_tr,
                tr_ids, movement_scaler, clinical_scaler,
                augmentation if not PAPER_PARITY else None,
                is_training=True, modality=modality
            )
            val_dataset = TrainingDataset(
                X_mov_train_val, X_quest_train_val, y_train_val,
                val_idx, movement_scaler, clinical_scaler,
                None, is_training=False, modality=modality
            )


            # --- Tell dataset which model is being trained (for augmentation policy) ---
            train_dataset.current_model_name = model_name
            val_dataset.current_model_name   = model_name

            train_dataset.paper_parity = PAPER_PARITY
            trainer_single_run.paper_parity = PAPER_PARITY

            # --- Instantiate model with BEST HPs ---
            # Prepare base parameters needed by all models
            params_for_instantiation = {'modality': modality, 'num_classes': 2}

            # Add modality-specific input sizes/features and model-specific hyperparameters
            if model_name == 'CA-LSTM-FCN':
                # Parity: BCE (1 logit) only for segment split
                n_classes = 1 if PAPER_PARITY else 2
                params_for_instantiation.update({
                    'input_channels': X_mov_train_val.shape[-1] if modality in ['movement_only', 'combined'] else 0,
                    'questionnaire_features': X_quest_train_val.shape[1] if modality in ['questionnaire_only', 'combined'] else 0,
                    'lstm_hidden_size': best_hps.get('lstm_hidden_size'),
                    'fcn_filters': best_hps.get('fcn_filters'),
                    'dropout': 0.3 if PAPER_PARITY else best_hps.get('dropout'),
                    'num_classes': n_classes
                })
                model = CA_LSTM_FCN(**params_for_instantiation)
            elif model_name == 'Transformer':
                 # Transformer uses config object
                 config = TimeSeriesTransformerConfig(
                      modality=modality,
                      num_classes=2,
                      input_channels=X_mov_train_val.shape[-1] if modality in ['movement_only', 'combined'] else 0,
                      questionnaire_features=X_quest_train_val.shape[1] if modality in ['questionnaire_only', 'combined'] else 0,
                      d_model=best_hps.get('d_model'),
                      nhead=best_hps.get('nhead'),
                      num_layers=best_hps.get('num_layers'),
                      dropout=best_hps.get('dropout')
                 )
                 model = TimeSeriesTransformer(config)
                 pass

            fold_metrics = trainer_single_run.train_single_fold(
                model, train_dataset, val_dataset, max_epochs=100, fold_num=fold_idx+1
            )

            # Collect all best-at-monitor metrics for this fold
            fold_scores_acc.append(fold_metrics['best_acc'])
            fold_scores_ba.append(fold_metrics['best_ba'])
            fold_scores_prec.append(fold_metrics['best_prec'])
            fold_scores_rec.append(fold_metrics['best_rec'])
            fold_scores_f1.append(fold_metrics['best_f1'])

            # single-line fold summary
            print(f"[Fold {fold_idx+1}] Acc={fold_metrics['best_acc']:.4f}  "
                  f"BA={fold_metrics['best_ba']:.4f}  Prec={fold_metrics['best_prec']:.4f}  "
                  f"Rec={fold_metrics['best_rec']:.4f}  F1={fold_metrics['best_f1']:.4f}  "
                  f"(best epoch {fold_metrics['best_epoch']})")

            if 'all_fold_metrics' not in locals():
                all_fold_metrics = []
            all_fold_metrics.append(fold_metrics)


            # cleanup
            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            else:
                torch.cuda.empty_cache()

        mean_acc  = float(np.mean(fold_scores_acc));  std_acc  = float(np.std(fold_scores_acc))
        mean_ba   = float(np.mean(fold_scores_ba));   std_ba   = float(np.std(fold_scores_ba))
        mean_prec = float(np.mean(fold_scores_prec)); std_prec = float(np.std(fold_scores_prec))
        mean_rec  = float(np.mean(fold_scores_rec));  std_rec  = float(np.std(fold_scores_rec))
        mean_f1   = float(np.mean(fold_scores_f1));   std_f1   = float(np.std(fold_scores_f1))

        print(f"\n  Results for {model_name} on {dataset_name}:")
        print(f"    Acc={mean_acc:.4f}±{std_acc:.4f}  BA={mean_ba:.4f}±{std_ba:.4f}")
        print(f"    Prec={mean_prec:.4f}±{std_prec:.4f}  Rec={mean_rec:.4f}±{std_rec:.4f}  F1={mean_f1:.4f}±{std_f1:.4f}")
        print(f"    Per-Fold metrics:")
        print(f"      Acc:  {np.round(fold_scores_acc, 4).tolist()}")
        print(f"      BA:   {np.round(fold_scores_ba, 4).tolist()}")
        print(f"      Prec: {np.round(fold_scores_prec, 4).tolist()}")
        print(f"      Rec:  {np.round(fold_scores_rec, 4).tolist()}")
        print(f"      F1:   {np.round(fold_scores_f1, 4).tolist()}")

        all_best_hp_results.append({
            'Dataset': dataset_name,
            'Model': model_name,
            'Hyperparameters': best_hps,
            'Mean_CV_Acc': mean_acc,  'Std_CV_Acc': std_acc,
            'Mean_CV_BA': mean_ba,    'Std_CV_BA': std_ba,
            'Mean_CV_Prec': mean_prec,'Std_CV_Prec': std_prec,
            'Mean_CV_Rec': mean_rec,  'Std_CV_Rec': std_rec,
            'Mean_CV_F1': mean_f1,    'Std_CV_F1': std_f1,
            'Fold_Acc_Scores': fold_scores_acc,
            'Fold_BA_Scores': fold_scores_ba,
            'Fold_Prec_Scores': fold_scores_prec,
            'Fold_Rec_Scores': fold_scores_rec,
            'Fold_F1_Scores': fold_scores_f1
        })

# --- Final Summary of Best HP Results ---
print("\n===== Summary of Evaluation with Best Hyperparameters =====")
if all_best_hp_results:
    best_hp_results_df = pd.DataFrame(all_best_hp_results)
    # Display results sorted by Mean CV Balanced Accuracy
    display(best_hp_results_df.sort_values('Mean_CV_BA', ascending=False))

    # Save these results to a separate summary file
    best_hp_results_df.to_csv(RESULTS_DIR / 'framework1_subjectlevel_results.csv', index=False)
    print(f"\nSummary saved to: {RESULTS_DIR / 'framework1_subjectlevel_results.csv'}")
else:
    print("No evaluation results were generated.")

Loaded HP results from: /content/drive/MyDrive/pads/results/evaluation_results_with_hp_tuning.csv
Models to process (in order): ['Transformer', 'CA-LSTM-FCN']

Starting evaluation with best hyperparameters...

--- Evaluating on Dataset: PD_VS_HC_MOVEMENT_ONLY ---

  Evaluating Model: Transformer
  Loaded/Using Hyperparameters: {'d_model': 128, 'nhead': 4, 'num_layers': 3, 'dropout': 0.2}
    Fold 1, Epoch   1: TrainLoss=0.6370 | ValLoss=0.5970 | Acc=0.775 | BA=0.500 | F1=0.873 | Prec=0.775 | Rec=1.000
    Fold 1, Epoch   2: TrainLoss=0.5828 | ValLoss=0.5983 | Acc=0.775 | BA=0.500 | F1=0.873 | Prec=0.775 | Rec=1.000
    Fold 1, Epoch   3: TrainLoss=0.5643 | ValLoss=0.5940 | Acc=0.775 | BA=0.500 | F1=0.873 | Prec=0.775 | Rec=1.000
    Fold 1, Epoch  21: TrainLoss=0.3193 | ValLoss=0.6438 | Acc=0.789 | BA=0.642 | F1=0.870 | Prec=0.833 | Rec=0.909
    Fold 1, Epoch  41: TrainLoss=0.2059 | ValLoss=0.6859 | Acc=0.817 | BA=0.793 | F1=0.876 | Prec=0.920 | Rec=0.836
    Fold 1, Epoch  61: TrainL

KeyboardInterrupt: 

### Framework 1 Result Analysis


In [16]:
# def analyze_results():
#     print("Analyzing Results:")
#     desired_order = [
#         'pd_vs_hc_movement_only',
#         'pd_vs_hc_questionnaire_only',
#         'pd_vs_hc_combined',
#         'pd_vs_dd_movement_only',
#         'pd_vs_dd_questionnaire_only',
#         'pd_vs_dd_combined'
#     ]

#     # Define the path for the output file
#     output_filename = "grouped_model_scores.txt"
#     output_filepath = RESULTS_DIR / output_filename

#     print(f"Saving grouped model scores to: {output_filepath}")

#     # Load the results from the new CSV file
#     new_results_csv_path = RESULTS_DIR / 'calstmfcn_segment_paperparity.csv'
#     try:
#         results_df_analysis = pd.read_csv(new_results_csv_path)
#         print(f"Loaded results from: {new_results_csv_path}")
#     except FileNotFoundError:
#         print(f"Error: Results CSV not found at {new_results_csv_path}. Cannot perform analysis.")
#         return {} # Return empty dictionary if file not found

#     # Reconstruct the all_results dictionary structure from the DataFrame
#     # Assuming the DataFrame has columns: 'Dataset', 'Model', 'Mean_CV_BA', 'Std_CV_BA'
#     all_results_analysis = {}
#     for index, row in results_df_analysis.iterrows():
#         dataset_name = row['Dataset']
#         model_name = row['Model']
#         mean_score = row['Mean_CV_BA']
#         std_score = row['Std_CV_BA']

#         if dataset_name not in all_results_analysis:
#             all_results_analysis[dataset_name] = {}

#         all_results_analysis[dataset_name][model_name] = {
#             'mean_cv_score': mean_score,
#             'std_cv_score': std_score,
#             # Add other relevant fields if needed, based on the original structure
#         }


#     analysis = {}

#     with open(output_filepath, 'w') as f:
#         f.write("Analyzing Results:\n")

#         for dataset_name in desired_order:
#             # Check if the dataset exists in the loaded results
#             if dataset_name in all_results_analysis:
#                 task_results = all_results_analysis[dataset_name]

#                 display_name = dataset_name.replace('_vs_', ' vs ').replace('_', ' ').title()
#                 print(f"\nAnalyzing {display_name}...")
#                 f.write(f"\nAnalyzing {display_name}...\n")

#                 best_model = None
#                 best_cv_score = -1.0

#                 # Iterate through models for the current dataset
#                 for model_name, result in task_results.items():
#                     cv_score = result['mean_cv_score']

#                     print(f"    {model_name}: {cv_score:.4f}")
#                     f.write(f"    {model_name}: {cv_score:.4f}\n")

#                     if cv_score > best_cv_score:
#                         best_cv_score = cv_score
#                         best_model = model_name
#         print("\nAnalysis complete!")
#         f.write("\nAnalysis complete!\n")
#     return analysis

# # Call the analyze_results function to perform the analysis and generate the output file
# analysis_results = analyze_results()

Analyzing Results:
Saving grouped model scores to: /content/drive/MyDrive/pads/results/grouped_model_scores.txt
Loaded results from: /content/drive/MyDrive/pads/results/calstmfcn_segment_paperparity.csv

Analyzing Pd Vs Hc Movement Only...
    CA-LSTM-FCN: 0.7905

Analyzing Pd Vs Hc Questionnaire Only...
    CA-LSTM-FCN: 0.9802

Analyzing Pd Vs Hc Combined...
    CA-LSTM-FCN: 0.9993

Analyzing Pd Vs Dd Movement Only...
    CA-LSTM-FCN: 0.7488

Analyzing Pd Vs Dd Questionnaire Only...
    CA-LSTM-FCN: 0.9890

Analyzing Pd Vs Dd Combined...
    CA-LSTM-FCN: 0.9991

Analysis complete!


## Framework 2: (Task-based Movement)

### Training and Evaluation


In [14]:
# === Transformer (movement-only), PD-vs-HC and PD-vs-DD ===
# Loads preprocessed /movement *.bin files, builds per-subject multi-task packs,
# trains with fusion across tasks

# ---------- CONFIG ----------
HP_CSV_PATH  = RESULTS_DIR / 'evaluation_results_with_hp_tuning.csv'

TASKS  = ["Relaxed1","Relaxed2","RelaxedTask1","RelaxedTask2","StretchHold",
          "HoldWeight","DrinkGlas","CrossArms","TouchNose","Entrainment1","Entrainment2"]
WRISTS = ["Left", "Right"]
SENSORS= ["Accelerometer","Gyroscope"]
AXES   = ["X","Y","Z"]
TARGET_T = 950
QUESTIONNAIRE_F = 30          # placeholder (unused in movement_only)
MAX_EPOCHS = 40
SCENARIO_EPOCHS = {"pd_vs_hc": 60, "pd_vs_dd": 60}
SEED = 42

# ---- EXPERIMENT KNOBS ----
# best tasks set
FOCUS_TASKS = ["HoldWeight","RelaxedTask2","Entrainment1","Relaxed1","StretchHold","TouchNose","DrinkGlas"]

# Choose the sensor fusion mode across wrists
#   "acc_gyro_both_wrists" -> [T, 12]
#   "acc_only_both_wrists" -> [T, 6]
#   "full_132" -> [T, 132]
EXPERIMENT_MODE = "acc_gyro_both_wrists"

# ---------- DEVICE / REPRO ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ---------- SMALL UTILS ----------
# ---- Augmentation helpers ----
class OnTheFlyAugmentation:
    """Rotate each 3-axis triplet; (optionally) time-warp. Works for C=6 or 12 or 132."""
    def __init__(self, rot_p=0.30, rot_deg=5, warp_p=0.0, warp_sigma=0.10, warp_knot=4):
        self.rot_p = float(rot_p)
        self.rot_deg = float(rot_deg)
        self.warp_p = float(warp_p)          # keep 0.0 to disable time warping
        self.warp_sigma = float(warp_sigma)
        self.warp_knot = int(warp_knot)

    def _rotation_matrix_3d(self, angle_deg):
        ang = np.radians(angle_deg)
        axis = np.random.choice(['x','y','z'])
        c, s = np.cos(ang), np.sin(ang)
        if axis == 'x':
            R = np.array([[1,0,0],[0,c,-s],[0,s,c]], dtype=np.float32)
        elif axis == 'y':
            R = np.array([[c,0,s],[0,1,0],[-s,0,c]], dtype=np.float32)
        else:
            R = np.array([[c,-s,0],[s,c,0],[0,0,1]], dtype=np.float32)
        return R

    def apply_axis_rotation(self, data, max_angle=15):
        """
        Rotate per wrist. If gyro is present, use the same R for that wrist's acc+gyro.
        data: [T, C] with C in {6, 12} given your EXPERIMENT_MODE.
        """
        x = data.copy()
        T, C = x.shape

        def rot_block(start, length, R):
            # rotate channels [start:start+length] interpreted as stacked 3D vectors
            assert length % 3 == 0
            for i in range(start, start + length, 3):
                x[:, i:i+3] = (R @ x[:, i:i+3].T).T

        if C == 12:
            # Left wrist (acc 0:3, gyro 3:6)
            R_L = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
            rot_block(0, 3, R_L)   # accL
            rot_block(3, 3, R_L)   # gyrL (same R as accL)

            # Right wrist (acc 6:9, gyro 9:12)
            R_R = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
            rot_block(6, 3, R_R)   # accR
            rot_block(9, 3, R_R)   # gyrR (same R as accR)

        elif C == 6:
            # Acc only, rotate each wrist independently
            R_L = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
            rot_block(0, 3, R_L)   # accL

            R_R = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
            rot_block(3, 3, R_R)   # accR
        elif C == 132:
            # 11 tasks × (Left/Right × Acc/Gyro × 3 axes) = 132
            stride = 12  # channels per task block
            # one R per wrist (consistent across all tasks, like 12-ch old path)
            R_L = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
            R_R = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
            for t_idx in range(len(TASKS)):
                base = t_idx * stride
                # Left wrist: acc (0:3), gyro (3:6)
                for off in (0, 3):
                    x[:, base+off:base+off+3] = (R_L @ x[:, base+off:base+off+3].T).T
                # Right wrist: acc (6:9), gyro (9:12)
                for off in (6, 9):
                    x[:, base+off:base+off+3] = (R_R @ x[:, base+off:base+off+3].T).T
        else:
            # Fallback: rotate each 3-axis triplet
            for s in range(0, C, 3):
                if s + 3 <= C:
                    R = self._rotation_matrix_3d(np.random.uniform(-max_angle, max_angle))
                    rot_block(s, 3, R)

        return x.astype(np.float32, copy=False)


    def apply_time_warping(self, data, sigma=0.2, knot=4):
        T, C = data.shape
        orig = np.arange(T)
        # smooth monotonic warp
        random_warps = np.abs(np.random.normal(loc=1.0, scale=sigma, size=(knot+2,)))
        time_warp = np.cumsum(random_warps)
        time_warp = time_warp / time_warp[-1] * (T - 1)
        warp_steps = np.linspace(0, T - 1, num=knot+2)
        idx = np.interp(orig, warp_steps, time_warp).clip(0, T - 1)

        y = np.empty_like(data)
        for i in range(C):
            y[:, i] = np.interp(idx, orig, data[:, i])
        return y.astype(np.float32, copy=False)

    def apply_random_augmentation(self, data):
        x = data.copy()
        if np.random.rand() < self.rot_p:
            x = self.apply_axis_rotation(x, max_angle=self.rot_deg)
        if np.random.rand() < self.warp_p:
            x = self.apply_time_warping(x, sigma=self.warp_sigma, knot=self.warp_knot)
        return x.astype(np.float32, copy=False)

def _clean_name(s):
    return re.sub(r"[^A-Za-z0-9]+", "", s)

def parse_hps(x):
    if not isinstance(x, str): return x
    try:
        return ast.literal_eval(x)
    except Exception:
        s = x.replace("None","null").replace("True","true").replace("False","false")
        if "'" in s and '"' not in s: s = re.sub(r"'", '"', s)
        return json.loads(s)

def _mode_input_channels():
    if EXPERIMENT_MODE == "acc_gyro_both_wrists": return 12
    if EXPERIMENT_MODE == "acc_only_both_wrists": return 6
    if EXPERIMENT_MODE == "full_132": return 132
    raise ValueError(f"Unsupported EXPERIMENT_MODE={EXPERIMENT_MODE} for late fusion")

def _sensor_label_for_results():
    return {
        "acc_gyro_both_wrists": "Accel+Gyro (BothWrists)",
        "acc_only_both_wrists": "Accelerometer (BothWrists)",
        "full_132": "AllTasks_AllSensors_BothWrists_132ch",
    }[EXPERIMENT_MODE].replace(" ", "").replace("(", "").replace(")", "")

def subject_id_from_filename(path):
    base = os.path.basename(path)
    m = re.findall(r"\d+", base)
    return int(m[0].lstrip("0") or "0") if m else None

def head_trim_to_T(x, target_T=950):
    """Trim from the START so we drop the waiting period, then pad at the END if needed."""
    T = x.shape[0]
    if T == target_T:
        return x
    if T > target_T:
        drop = T - target_T
        return x[drop:, :][:target_T]
    # shorter -> pad at the END
    pad = target_T - T
    return np.pad(x, ((0, pad), (0, 0)), mode="constant")

def load_subject_bin_multiindex(path, TASKS, WRISTS, SENSORS, AXES):
    arr = np.fromfile(path, dtype=np.float32)
    assert arr.size % 132 == 0, f"Unexpected float count in {os.path.basename(path)}"
    T = arr.size // 132
    data = arr.reshape(T, 132)  # [T, C]
    cols = pd.MultiIndex.from_product([TASKS, WRISTS, SENSORS, AXES],
                                      names=["task","wrist","sensor","axis"])
    df = pd.DataFrame(data, columns=cols)
    return df

def load_full_label_map():
    assert os.path.exists(LABELS_CSV), f"Missing LABELS_CSV: {LABELS_CSV}"
    df = pd.read_csv(LABELS_CSV)
    assert {"id","label"}.issubset(df.columns), "file_list.csv must contain 'id' and 'label'."
    df = df.copy()
    df["id"] = df["id"].astype(str).str.extract(r"(\d+)").astype(int)
    lab = dict(zip(df["id"].astype(int), df["label"].astype(int)))
    print(f"[info] Loaded label map for {len(lab)} subjects.")
    return lab

hp_df = pd.read_csv(HP_CSV_PATH)
CSV_ALIASES = {
    "Transformer":   ["Transformer","Transformer-EarlyFusion"],
    "CA_LSTM_FCN":   ["CA_LSTM_FCN","CA-LSTM-FCN","CALSTMFCN","CA_LSTM+FCN"]
}
def best_hps_for(model_name, scenario_substr=None):
    names = CSV_ALIASES.get(model_name, [model_name])
    df = hp_df[hp_df["Model"].isin(names)]
    if scenario_substr:
        dff = df[df["Dataset"].str.contains(scenario_substr, case=False, na=False)]
        if not dff.empty:
            df = dff
    if df.empty:
        # No tuned HPs for this model/scenario -> use model defaults
        return {}
    row = df.sort_values(by="Mean_CV_Score", ascending=False).iloc[0]
    return parse_hps(row["Best_Hyperparameters"])

def make_class_weights(y_train: np.ndarray, num_classes: int = 2) -> torch.Tensor:
    counts = np.bincount(y_train, minlength=num_classes).astype(float)
    counts[counts == 0] = 1.0
    inv = 1.0 / counts
    w = inv / inv.sum() * num_classes
    return torch.tensor(w, dtype=torch.float32)

# ----------- MODEL BUILDER -----------
def build_model_transformer(hps, modality="movement_only"):
    in_ch = _mode_input_channels()
    tuned_hps = dict(hps or {})
    if modality == "movement_only":
        # slightly beefier defaults for 132ch
        tuned_hps.setdefault("d_model", 192 if in_ch == 132 else 128)
        tuned_hps.setdefault("nhead", 6 if in_ch == 132 else 4)
        tuned_hps.setdefault("num_layers", 4)
        tuned_hps.setdefault("dropout", 0.20)
    tuned_hps.setdefault("num_classes", 2)
    cfg = TimeSeriesTransformerConfig(
        modality=modality,
        sequence_length=TARGET_T,
        input_channels=in_ch,
        questionnaire_features=QUESTIONNAIRE_F,
        **tuned_hps
    )
    return TimeSeriesTransformer(cfg)

def build_model_ca_lstm_fcn(hps, modality="movement_only"):
    in_ch = _mode_input_channels()
    tuned = {
        "input_channels": in_ch,
        "sequence_length": TARGET_T,
        "num_classes": 2,
        "questionnaire_features": QUESTIONNAIRE_F,
        "lstm_hidden_size": hps.get("lstm_hidden_size", 128),
        "fcn_filters": hps.get("fcn_filters", [128, 256, 128]),
        "dropout": hps.get("dropout", 0.2),
        "modality": modality,
    }
    return CA_LSTM_FCN(**tuned)

# ----------- DATA PACKING -----------
def build_per_subject_taskpack(subjects, label_map, task_set):
    packs = []
    for sid, p in subjects:
        if sid not in label_map:
            continue
        df = load_subject_bin_multiindex(p, TASKS, WRISTS, SENSORS, AXES)
        y  = int(label_map[sid])

        # --- PAPER / 132ch path: one block per subject ---
        if EXPERIMENT_MODE == "full_132":
            X = df.to_numpy().astype(np.float32)
            X = head_trim_to_T(X, TARGET_T)
            if not np.isnan(X).any():
                packs.append({"sid": sid, "y": y, "tasks": {"ALL": X}})
            continue

        # --- Original 6/12-ch late-fusion path ---
        task_dict = {}
        for task in task_set:
            if EXPERIMENT_MODE == "acc_gyro_both_wrists":
                accL = df[(task, "Left",  "Accelerometer")].to_numpy()
                gyrL = df[(task, "Left",  "Gyroscope")].to_numpy()
                accR = df[(task, "Right", "Accelerometer")].to_numpy()
                gyrR = df[(task, "Right", "Gyroscope")].to_numpy()
                X = np.concatenate([accL, gyrL, accR, gyrR], axis=1)  # [T,12]
            elif EXPERIMENT_MODE == "acc_only_both_wrists":
                accL = df[(task, "Left",  "Accelerometer")].to_numpy()
                accR = df[(task, "Right", "Accelerometer")].to_numpy()
                X = np.concatenate([accL, accR], axis=1)              # [T,6]
            else:
                raise ValueError("Unexpected mode")
            X = head_trim_to_T(X, TARGET_T).astype(np.float32)
            if not np.isnan(X).any():
                task_dict[task] = X

        if len(task_dict) > 0:
            packs.append({"sid": sid, "y": y, "tasks": task_dict})
    return packs


# ----------- DATASETS -----------
class MultiTaskDataset(Dataset):
    """
    For training: sample_one_task=True -> (x[T,C], zeros, y) with optional augmentation.
    For eval:     sample_one_task=False -> (x_stack[num_tasks,T,C], mask[num_tasks], y)
    """
    def __init__(self, subject_packs, task_list, sample_one_task: bool,
                 augmentor=None, is_training: bool=False, aug_prob: float=0.35):
        self.packs = subject_packs
        self.task_list = list(task_list)  # fixed order for stacking
        self.sample_one = bool(sample_one_task)
        self.augmentor = augmentor
        self.is_training = bool(is_training)
        self.aug_prob = float(aug_prob)

    def __len__(self): return len(self.packs)

    def __getitem__(self, idx):
        pack = self.packs[idx]
        y = int(pack["y"])

        if self.sample_one:
            t = next(iter(pack["tasks"])) if len(pack["tasks"]) == 1 else random.choice(list(pack["tasks"].keys()))
            x = pack["tasks"][t]

            if self.is_training and (self.augmentor is not None) and (self.aug_prob > 0) \
              and (np.random.rand() < self.aug_prob) \
              and hasattr(self.augmentor, "apply_random_augmentation"):
                x = self.augmentor.apply_random_augmentation(x)

            return torch.from_numpy(x), torch.zeros(QUESTIONNAIRE_F, dtype=torch.float32), y

        # eval: stack all tasks (no augmentation)
        xs, mask = [], []
        for t in self.task_list:
            if t in pack["tasks"]:
                xs.append(torch.from_numpy(pack["tasks"][t]))
                mask.append(1.0)
            else:
                xs.append(torch.zeros(TARGET_T, _mode_input_channels(), dtype=torch.float32))
                mask.append(0.0)
        x_stack = torch.stack(xs, dim=0)
        mask = torch.tensor(mask, dtype=torch.float32)
        return x_stack, mask, y

# ----------- TRAINER -----------
class LateFusionTrainer:
    def __init__(self, device="cpu", class_weights=None, label_smoothing=0.05, lr=5e-4, verbose_every=1):
        self.device = device
        self.class_weights = class_weights
        self.label_smoothing = float(label_smoothing)
        self.lr = float(lr)
        self.verbose_every = int(verbose_every)

    def _criterion(self):
        alpha = self.class_weights.to(self.device) if self.class_weights is not None else None
        return nn.CrossEntropyLoss(weight=alpha, label_smoothing=self.label_smoothing)

    def train_single_fold(self, model, train_loader, val_loader, max_epochs, fold_num):
        model = model.to(self.device)
        opt = optim.AdamW(model.parameters(), lr=self.lr, weight_decay=5e-4)
        sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=max_epochs)
        crit = self._criterion()

        best_state = None
        patience = 30
        bad = 0
        best_pack = {"ba": 0.0, "acc": 0.0, "prec": 0.0, "rec": 0.0, "f1": 0.0}

        for epoch in range(max_epochs):
            # ----------------- TRAIN -----------------
            model.train()
            train_loss_sum = 0.0
            train_batches = 0
            for x, _, y in train_loader:  # (x[T,C], zeros, y)
                x = x.to(self.device, dtype=torch.float32)          # [B,T,C]
                y_t = torch.as_tensor(y, device=self.device)

                opt.zero_grad()
                zeros_q = torch.zeros(x.size(0), QUESTIONNAIRE_F, device=self.device)
                logits = model(x, zeros_q)                           # [B,num_classes]
                loss = crit(logits, y_t.long())
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                opt.step()

                train_loss_sum += float(loss.detach().cpu().item())
                train_batches += 1

            # ----------------- VALID -----------------
            model.eval()
            y_true, y_pred = [], []
            val_loss_sum = 0.0
            val_batches = 0
            with torch.no_grad():
                for x_stack, mask, y in val_loader:
                    B, K, _, _ = x_stack.shape
                    x_stack = x_stack.to(self.device, dtype=torch.float32)
                    mask = mask.to(self.device, dtype=torch.float32)
                    y_t = torch.as_tensor(y, device=self.device)

                    logits_tasks = []
                    active_idx = []
                    for k in range(K):
                        if mask[:, k].sum().item() == 0:
                            continue
                        xk = x_stack[:, k, :, :]
                        zeros_q = torch.zeros(B, QUESTIONNAIRE_F, device=self.device)
                        lk = model(xk, zeros_q)                      # [B,C]
                        logits_tasks.append(lk.unsqueeze(1))         # [B,1,C]
                        active_idx.append(k)

                    if not logits_tasks:
                        continue

                    logits_tasks = torch.cat(logits_tasks, dim=1)    # [B,K',C]
                    mask_active = mask[:, active_idx]                 # [B,K']

                    gate = logits_tasks.mean(-1)                      # [B,K']
                    gate = gate.masked_fill(mask_active == 0, float('-inf'))
                    att  = torch.softmax(gate, dim=1)                 # [B,K']
                    fused = (logits_tasks * att.unsqueeze(-1)).sum(dim=1)  # [B,C]

                    # accumulate val loss on fused logits
                    vloss = crit(fused, y_t.long())
                    val_loss_sum += float(vloss.detach().cpu().item())
                    val_batches += 1

                    preds = fused.argmax(dim=1)
                    y_pred.extend(preds.detach().cpu().numpy().tolist())
                    y_true.extend(y.detach().cpu().numpy().tolist())

            # Metrics
            y_true_np = np.array(y_true)
            y_pred_np = np.array(y_pred)
            ba = balanced_accuracy_score(y_true_np, y_pred_np) if len(y_true_np) else 0.0
            acc = (y_true_np == y_pred_np).mean() if len(y_true_np) else 0.0
            prec = precision_score(y_true_np, y_pred_np, zero_division=0) if len(y_true_np) else 0.0
            rec  = recall_score(y_true_np, y_pred_np, zero_division=0) if len(y_true_np) else 0.0
            f1   = f1_score(y_true_np, y_pred_np, zero_division=0) if len(y_true_np) else 0.0

            sch.step()

            # Early stopping on BA; remember all metrics at the best-BA epoch
            if ba > best_pack["ba"]:
                best_pack.update({"ba": ba, "acc": acc, "prec": prec, "rec": rec, "f1": f1})
                bad = 0
                best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            else:
                bad += 1

            # ---- Logging cadence ----
            if (epoch < 3) or ((epoch + 1) % self.verbose_every == 0):
                tl = train_loss_sum / max(1, train_batches)
                vl = val_loss_sum / max(1, val_batches)
                print(
                    f"    Fold {fold_num}, Epoch {epoch+1:3d}: "
                    f"TrainLoss={tl:.4f} | ValLoss={vl:.4f} | "
                    f"BalAcc={ba:.4f} | Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | F1={f1:.4f}"
                )

            if bad >= patience:
                print(f"    Early stopping at epoch {epoch+1}")
                break

        if best_state is not None:
            model.load_state_dict(best_state)
        return best_pack["ba"], best_pack["acc"], best_pack["prec"], best_pack["rec"], best_pack["f1"]

# ----------- MAIN SCENARIO RUN  -----------
def run_scenario_latefusion(scenario_name, label_keep, label_remap, hp_scenario_substr, task_set,model_name="Transformer"):
    print(f"\n===== Scenario: {scenario_name} — TaskSet: {len(task_set)} tasks =====")
    # --- precompute names used by both CSV and ckpt paths ---
    sensor_label = _sensor_label_for_results()
    task_tag = (
        "besttasks" if set(task_set) == set(FOCUS_TASKS)
        else ("alltasks" if set(task_set) == set(TASKS) else f"{len(task_set)}tasks")
    )

    # Discover subjects
    bin_paths = sorted(glob.glob(os.path.join(MOVEMENT_DIR, "*.bin")))
    assert bin_paths, f"No .bin files found in {MOVEMENT_DIR}"
    full_label_map = load_full_label_map()

    # Filter & remap labels
    subjects = []
    for p in bin_paths:
        sid = subject_id_from_filename(p)
        if sid is None or sid not in full_label_map: continue
        lbl = full_label_map[sid]
        if lbl in label_keep:
            subjects.append((sid, p))
    assert subjects, f"No subjects with labels in {label_keep} found."

    label_map = {sid: label_remap[full_label_map[sid]] for sid, _ in subjects}

    # Build per-subject packs (selected tasks only)
    packs = build_per_subject_taskpack(subjects, label_map, task_set)

    # Convert to arrays for CV split (subject-level, stratified by label)
    y_all = np.array([p["y"] for p in packs], dtype=int)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

    # HPs for Transformer
    hps = best_hps_for(model_name, scenario_substr=hp_scenario_substr)
    if (model_name == "Transformer") and (scenario_name == "pd_vs_dd"):
        hps = {**hps, "d_model": hps.get("d_model", 128), "nhead": hps.get("nhead", 4),
              "num_layers": max(4, hps.get("num_layers", 3)), "dropout": max(0.20, hps.get("dropout", 0.1))}
    print("[HPs]", scenario_name, model_name + ":", hps)

     # choose the right model builder
    builders = {
        "Transformer":   lambda: build_model_transformer(hps, modality="movement_only"),
        "CA_LSTM_FCN":   lambda: build_model_ca_lstm_fcn(hps, modality="movement_only"),
    }

    fold_ba_scores = []
    fold_acc_scores = []
    fold_prec_scores, fold_rec_scores, fold_f1_scores = [], [], []
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(y_all)), y_all), start=1):
        # build train/val subject lists
        train_packs = [packs[i] for i in train_idx]
        val_packs   = [packs[i] for i in val_idx]

        # standardize movement per fold (fit on ALL train samples across tasks)
        # we do per-channel standardization.
        # gather all train arrays
        all_train = []
        for sp in train_packs:
            for t, X in sp["tasks"].items():
                all_train.append(X)
        all_train = np.stack(all_train, axis=0)  # [M, T, C]
        scaler = StandardScaler().fit(all_train.reshape(-1, all_train.shape[-1]))

        # apply scaler (in-place transform copies)
        def _apply_scaler(packs_list):
            out = []
            for sp in packs_list:
                new_tasks = {}
                for t, X in sp["tasks"].items():
                    Xs = scaler.transform(X)  # [T,C]
                    new_tasks[t] = Xs.astype(np.float32)
                out.append({"sid": sp["sid"], "y": sp["y"], "tasks": new_tasks})
            return out

        train_packs_s = _apply_scaler(train_packs)
        val_packs_s   = _apply_scaler(val_packs)

        # -------- Datasets / Loaders --------
        # ----- choose task list for the dataset -----
        eval_task_list = ["ALL"] if EXPERIMENT_MODE == "full_132" else task_set

        # ----- augmentation selection -----
        aug_for_train = OnTheFlyAugmentation(
            rot_p=0.25 if scenario_name == "pd_vs_dd" else 0.35,
            rot_deg=5   if scenario_name == "pd_vs_dd" else 8,
            warp_p=0.0
        )
        aug_prob = 0.30 if scenario_name == "pd_vs_dd" else 0.35


        train_ds = MultiTaskDataset(
            train_packs_s, task_list=eval_task_list, sample_one_task=True,
            augmentor=aug_for_train, is_training=True, aug_prob=aug_prob
        )
        val_ds = MultiTaskDataset(
            val_packs_s, task_list=eval_task_list, sample_one_task=False,
            augmentor=None, is_training=False
        )

        bs = 8 if (model_name == "Transformer" and EXPERIMENT_MODE == "full_132") else 16
        g = torch.Generator(); g.manual_seed(SEED)
        train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, drop_last=False, generator=g)
        val_loader   = DataLoader(val_ds,   batch_size=bs, shuffle=False, drop_last=False)

        # Build model & trainer
        model = builders[model_name]()

        class_w = make_class_weights(np.array([p["y"] for p in train_packs_s], dtype=int), num_classes=2)

        trainer = LateFusionTrainer(device=device, class_weights=class_w, label_smoothing=0.05,
                                  lr=5e-4)

        epochs = SCENARIO_EPOCHS.get(scenario_name, MAX_EPOCHS)
        num_tasks_for_log = 1 if EXPERIMENT_MODE == "full_132" else len(task_set)

        print(f"\n=== [{scenario_name}] TaskFusion[{num_tasks_for_log}] — {model_name} : "
              f"Fold {fold_idx}/5, epochs={epochs} ===")

        best_ba, best_acc, best_prec, best_rec, best_f1 = trainer.train_single_fold(
            model, train_loader, val_loader, max_epochs=epochs, fold_num=fold_idx
        )
        fold_ba_scores.append(float(best_ba))
        fold_acc_scores.append(float(best_acc))
        fold_prec_scores.append(float(best_prec))
        fold_rec_scores.append(float(best_rec))
        fold_f1_scores.append(float(best_f1))

        # save the best FP32 checkpoint for this fold
        ckpt_path = os.path.join(
            RESULTS_DIR,
            f"ckpt_{scenario_name}_{_sensor_label_for_results()}_{_clean_name(model_name)}_{task_tag}_fold{fold_idx}.pth"
        )
        torch.save(model.state_dict(), ckpt_path)
        print(f"[ckpt] Saved best FP32 weights -> {ckpt_path}")

        del model
        if torch.cuda.is_available(): torch.cuda.empty_cache()

    # aggregate both metrics
    mean_ba  = float(np.mean(fold_ba_scores));  std_ba  = float(np.std(fold_ba_scores,  ddof=1))
    mean_acc = float(np.mean(fold_acc_scores)); std_acc = float(np.std(fold_acc_scores, ddof=1))
    mean_prec = float(np.mean(fold_prec_scores)); std_prec = float(np.std(fold_prec_scores, ddof=1))
    mean_rec  = float(np.mean(fold_rec_scores)); std_rec = float(np.std(fold_rec_scores,  ddof=1))
    mean_f1   = float(np.mean(fold_f1_scores)); std_f1 = float(np.std(fold_f1_scores,   ddof=1))

    num_tasks_meta = 1 if EXPERIMENT_MODE == "full_132" else len(task_set)
    print(
        f">>> [{scenario_name}] TaskFusion[{num_tasks_meta}] — {model_name}:"
        f" BA={mean_ba:.3f}±{std_ba:.3f} | Acc={mean_acc:.3f}±{std_acc:.3f}"
        f" | Prec={mean_prec:.3f}±{std_prec:.3f} | Rec={mean_rec:.3f}±{std_rec:.3f} | F1={mean_f1:.3f}±{std_f1:.3f}\n"
        f"    BA per-fold={np.round(fold_ba_scores,3).tolist()}, Acc per-fold={np.round(fold_acc_scores,3).tolist()}"
    )

    out_csv = os.path.join(
        RESULTS_DIR,
        f"subjectfusion_{scenario_name}_{sensor_label}_{_clean_name(model_name)}_{task_tag}.csv"
    )

    num_tasks_meta = 1 if EXPERIMENT_MODE == "full_132" else len(task_set)
    # single-row CSV with both BA and Acc
    df_out = pd.DataFrame([{
        "Scenario": scenario_name,
        "TaskSet": task_tag,
        "NumTasks": num_tasks_meta,
        "Sensor": sensor_label,
        "Model": model_name,
        "Val Balanced Acc (mean)": mean_ba,
        "Val Balanced Acc (std)": std_ba,
        "Val Accuracy (mean)": mean_acc,
        "Val Accuracy (std)": std_acc,
        "Val Precision (mean)": mean_prec,
        "Val Precision (std)": std_prec,
        "Val Recall (mean)": mean_rec,
        "Val Recall (std)": std_rec,
        "Val F1 (mean)": mean_f1,
        "Val F1 (std)": std_f1,
        "Per-fold BalAcc": fold_ba_scores,
        "Per-fold Acc": fold_acc_scores,
        "Per-fold Precision": fold_prec_scores,
        "Per-fold Recall": fold_rec_scores,
        "Per-fold F1": fold_f1_scores,
        "N subjects": int(len(packs))
    }])

    df_out.to_csv(out_csv, index=False)
    print(f"Saved final results to: {out_csv}")
    return df_out


# ===================== RUNS =====================
# 1) PD vs HC with BEST tasks
# df_hc_best = run_scenario_latefusion(
#     scenario_name="pd_vs_hc",
#     label_keep={0,1},
#     label_remap={0:0, 1:1},
#     hp_scenario_substr="pd_vs_hc",
#     task_set=FOCUS_TASKS,
#     model_name="CA_LSTM_FCN"
# )

# 2) PD vs HC with ALL tasks
df_hc_all = run_scenario_latefusion(
    scenario_name="pd_vs_hc",
    label_keep={0,1},
    label_remap={0:0, 1:1},
    hp_scenario_substr="pd_vs_hc",
    task_set=TASKS,
    model_name="Transformer"
)

# 3) PD vs DD with BEST tasks
# df_dd_best = run_scenario_latefusion(
#     scenario_name="pd_vs_dd",
#     label_keep={1,2},
#     label_remap={1:1, 2:0},
#     hp_scenario_substr="pd_vs_dd|pd_vs_omd",
#     task_set=FOCUS_TASKS,
#     model_name="CA_LSTM_FCN"
# )

# 4) PD vs DD with ALL tasks
df_dd_all = run_scenario_latefusion(
    scenario_name="pd_vs_dd",
    label_keep={1,2},
    label_remap={1:1, 2:0},
    hp_scenario_substr="pd_vs_dd|pd_vs_omd",
    task_set=TASKS,
    model_name="Transformer")


===== Scenario: pd_vs_hc — TaskSet: 11 tasks =====
[info] Loaded label map for 469 subjects.
[HPs] pd_vs_hc Transformer: {'d_model': 256, 'nhead': 8, 'num_layers': 2, 'dropout': 0.2}

=== [pd_vs_hc] TaskFusion[11] — Transformer : Fold 1/5, epochs=60 ===


KeyboardInterrupt: 

### Transformer Model Quantization


In [21]:
# === ONNX dynamic quant (movement-only Transformer) ===
ckpts = glob.glob(os.path.join(RESULTS_DIR, "ckpt_pd_vs_*_fold*.pth"))
print("Found ckpts:", len(ckpts))
assert len(ckpts) >= 10, "Expected ckpt files missing. Run training first."
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
EXPERIMENT_MODE = "acc_gyro_both_wrists" #all sensors, all wrists, all tasks (use this mode as a default)

def strip_value_info(onnx_path: str):
    m = onnx.load(onnx_path)
    del m.graph.value_info[:]
    for n in m.graph.node:
        n.doc_string = ""
    onnx.save(m, onnx_path)

def _mode_input_channels():
    return 12 if EXPERIMENT_MODE=="acc_gyro_both_wrists" else (6 if EXPERIMENT_MODE=="acc_only_both_wrists" else 132)

def _sensor_label_for_results():
    return {
        "acc_gyro_both_wrists": "Accel+GyroBothWrists",
        "acc_only_both_wrists": "AccelerometerBothWrists",
        "full_132": "AllTasks_AllSensors_BothWrists_132ch",
    }[EXPERIMENT_MODE]

def sizeof_mb(path):
    return os.path.getsize(path) / (1024*1024)

class _ExportWrapper(nn.Module):
    def __init__(self, model): super().__init__(); self.model = model.eval()
    def forward(self, movement_data): return self.model(movement_data, None)

def export_onnx_1input(model, onnx_path, T, C, opset=18):
    wrapper = _ExportWrapper(copy.deepcopy(model).to("cpu").eval())

    dummy = torch.randn(1, T, C, dtype=torch.float32)

    torch.onnx.export(
        wrapper,
        dummy,
        onnx_path,
        input_names=["movement_data"],
        output_names=["logits"],
        opset_version=opset,
        do_constant_folding=True,
    )
from onnxruntime.quantization import quantize_dynamic, QuantType
import inspect
print(inspect.signature(quantize_dynamic))

def quantize_onnx_dynamic(fp32_path, int8_path):
    quantize_dynamic(
        model_input=fp32_path,
        model_output=int8_path,
        op_types_to_quantize=["MatMul","Gemm"],
        per_channel=True,
        reduce_range=False,
        weight_type=QuantType.QInt8,
        extra_options={"DisableShapeInference": True},
    )
    print("[quant] DisableShapeInference=True")


@torch.no_grad()
def eval_bal_acc_onnx(sess, val_ds):
    inp = sess.get_inputs()[0].name
    y_true, y_pred = [], []
    for x_stack, task_mask, y in DataLoader(val_ds, batch_size=1, shuffle=False):
        K = x_stack.shape[1]
        outs = []
        for k in range(K):
            if task_mask[:, k].sum().item() == 0: continue
            xk = x_stack[:, k, :, :].numpy().astype(np.float32)
            outs.append(sess.run(None, {inp: xk})[0])      # [1,C]
        if not outs: continue
        lt = np.concatenate([o[:,None,:] for o in outs], 1)  # [1,K',C]
        gate = lt.mean(-1); att = np.exp(gate - gate.max(1, keepdims=True))
        att /= att.sum(1, keepdims=True)
        fused = (lt * att[..., None]).sum(1)  # [1,C]
        y_pred.append(int(fused.argmax(-1)[0])); y_true.append(int(y.item()))
    return balanced_accuracy_score(np.array(y_true), np.array(y_pred))

@torch.no_grad()
def bench_latency_ms_onnx(sess, val_ds, warmup=3, iters=30):
    inp = sess.get_inputs()[0].name
    x_stack, task_mask, _ = next(iter(DataLoader(val_ds, batch_size=1, shuffle=False)))
    K = x_stack.shape[1]
    for _ in range(max(0,warmup)):
        for k in range(K):
            if task_mask[:,k].sum()==0: continue
            _ = sess.run(None, {inp: x_stack[:,k,:,:].numpy().astype(np.float32)})
    t0 = time.perf_counter()
    for _ in range(max(1,iters)):
        for k in range(K):
            if task_mask[:,k].sum()==0: continue
            _ = sess.run(None, {inp: x_stack[:,k,:,:].numpy().astype(np.float32)})
    return (time.perf_counter()-t0)*1000/max(1,iters)

def build_val_dataset_for_fold(subjects, label_map, task_set, fold_idx, n_splits=5):
    packs = build_per_subject_taskpack(subjects, label_map, task_set)
    y_all = np.array([p["y"] for p in packs], dtype=int)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    train_idx, val_idx = list(skf.split(np.zeros(len(y_all)), y_all))[fold_idx-1]
    train_packs = [packs[i] for i in train_idx]; val_packs = [packs[i] for i in val_idx]
    # fit scaler on all train arrays
    all_train = np.stack([X for sp in train_packs for X in sp["tasks"].values()], axis=0)
    scaler = StandardScaler().fit(all_train.reshape(-1, all_train.shape[-1]))
    def _apply(packs):
        out = []
        for sp in packs:
            ts = {t: scaler.transform(X).astype(np.float32) for t, X in sp["tasks"].items()}
            out.append({"sid": sp["sid"], "y": sp["y"], "tasks": ts})
        return out
    train_s, val_s = _apply(train_packs), _apply(val_packs)
    eval_tasks = ["ALL"] if EXPERIMENT_MODE=="full_132" else task_set
    val_ds = MultiTaskDataset(val_s, task_list=eval_tasks, sample_one_task=False, augmentor=None, is_training=False)
    return val_ds, len(packs)

def run_quant_for_model_onnx(scenario_name, label_keep, label_remap, task_set):
    print(f"\n=== Quantization (ONNX): Transformer — {scenario_name} — mode={EXPERIMENT_MODE} ===")
    # subjects
    full_map = load_full_label_map()
    subs = []
    for p in sorted(glob.glob(os.path.join(MOVEMENT_DIR, "*.bin"))):
        sid = subject_id_from_filename(p)
        if sid is not None and sid in full_map and full_map[sid] in label_keep:
            subs.append((sid, p))
    assert subs, f"No subjects for {scenario_name}"
    label_map = {sid: label_remap[full_map[sid]] for sid,_ in subs}

    sensor = _sensor_label_for_results()
    # Keep filenames consistent with training for full_132
    if EXPERIMENT_MODE == "full_132":
        task_tag = "alltasks"
    else:
        task_tag = "alltasks" if set(task_set) == set(TASKS) else f"{len(task_set)}tasks"

    rows = []; C = _mode_input_channels()

    for fold_idx in range(1,6):
        val_ds, n_subj = build_val_dataset_for_fold(subs, label_map, task_set, fold_idx)
        # rebuild model + load state
        try:
            hps = best_hps_for("Transformer", scenario_substr=("pd_vs_dd|pd_vs_omd" if scenario_name=="pd_vs_dd" else "pd_vs_hc"))
        except Exception:
            hps = {}
        if scenario_name == "pd_vs_dd":
            hps = {**hps, "d_model": hps.get("d_model", 128), "nhead": hps.get("nhead", 4),
                   "num_layers": max(4, hps.get("num_layers", 3)), "dropout": max(0.20, hps.get("dropout", 0.1))}
        model = build_model_transformer(hps, modality="movement_only")
        ckpt = os.path.join(RESULTS_DIR, f"ckpt_{scenario_name}_{sensor}_{_clean_name('Transformer')}_{task_tag}_fold{fold_idx}.pth")
        state = torch.load(ckpt, map_location="cpu"); model.load_state_dict(state, strict=True); model.eval()

        # export & quantize
        onnx_fp32 = os.path.join(RESULTS_DIR, f"onnx_{scenario_name}_{sensor}_Transformer_{task_tag}_fold{fold_idx}.onnx")
        export_onnx_1input(model, onnx_fp32, T=TARGET_T, C=C)
        # remove conflicting shape metadata before quantization
        strip_value_info(onnx_fp32)
        onnx_int8 = onnx_fp32.replace(".onnx", "_int8.onnx")
        quantize_onnx_dynamic(onnx_fp32, onnx_int8)

        # run ORT
        s_fp32 = ort.InferenceSession(onnx_fp32, providers=["CPUExecutionProvider"])
        s_int8 = ort.InferenceSession(onnx_int8, providers=["CPUExecutionProvider"])
        ba_f  = eval_bal_acc_onnx(s_fp32, val_ds); lat_f  = bench_latency_ms_onnx(s_fp32, val_ds)
        ba_q  = eval_bal_acc_onnx(s_int8, val_ds); lat_q  = bench_latency_ms_onnx(s_int8, val_ds)
        sz_f, sz_q = sizeof_mb(onnx_fp32), sizeof_mb(onnx_int8)

        print(f"  Fold {fold_idx}: BA {ba_f:.3f}→{ba_q:.3f} | ms {lat_f:.2f}→{lat_q:.2f} | MB {sz_f:.1f}→{sz_q:.1f}")
        rows.append({
            "Scenario": scenario_name, "Model": "Transformer", "Mode": EXPERIMENT_MODE, "Fold": fold_idx,
            "Quant Strategy": "ONNX dynamic (MatMul/Gemm)", "N subjects": int(n_subj),
            "BA FP32": float(ba_f), "BA INT8": float(ba_q), "Δ BA": float(ba_q - ba_f),
            "Latency FP32 (ms)": float(lat_f), "Latency INT8 (ms)": float(lat_q),
            "Latency Δ% (↓=faster)": float(100.0*(lat_f - lat_q)/max(lat_f,1e-8)),
            "Size FP32 (MB)": float(sz_f), "Size INT8 (MB)": float(sz_q)
        })
    df = pd.DataFrame(rows)
    out_csv = os.path.join(RESULTS_DIR, f"quant_report_{scenario_name}_{sensor}_Transformer_{task_tag}_onnx.csv")
    df.to_csv(out_csv, index=False); print(f"[saved] {out_csv}")
    return df

# ---- Run both scenarios ----
SCENARIOS = [
    dict(name="pd_vs_hc", keep={0,1}, remap={0:0, 1:1}),
    dict(name="pd_vs_dd", keep={1,2}, remap={1:1, 2:0}),
]
if EXPERIMENT_MODE == "full_132":
    TASK_SET = TASKS
else:
    TASK_SET = TASKS if EXPERIMENT_MODE in ("acc_gyro_both_wrists", "acc_only_both_wrists") else ["ALL"]

for sc in SCENARIOS:
    run_quant_for_model_onnx(sc["name"], sc["keep"], sc["remap"], TASK_SET)


Found ckpts: 20
(model_input: 'str | Path | onnx.ModelProto', model_output: 'str | Path', op_types_to_quantize=None, per_channel=False, reduce_range=False, weight_type=<QuantType.QInt8: 0>, nodes_to_quantize=None, nodes_to_exclude=None, use_external_data_format=False, extra_options=None)

=== Quantization (ONNX): Transformer — pd_vs_hc — mode=acc_gyro_both_wrists ===
[info] Loaded label map for 469 subjects.
[torch.onnx] Obtain model graph for `_ExportWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `_ExportWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 2 of general pattern rewrite rules.


[quant] DisableShapeInference=True
  Fold 1: BA 0.820→0.762 | ms 211.31→164.16 | MB 7.3→2.7
[torch.onnx] Obtain model graph for `_ExportWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `_ExportWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 2 of general pattern rewrite rules.


[quant] DisableShapeInference=True


KeyboardInterrupt: 